In [1]:
import numpy as np
from scipy.spatial.distance import jaccard, cosine
from bs4 import BeautifulSoup
import re
import requests
import openai
import os
import json
from datetime import datetime
import pandas as pd
import tqdm
from matplotlib import pyplot as plt
import glob as glob

In [4]:
my_dict = {'a': 1, 'b': 2}
other_dict = {'c': 3, 'd': 4}
my_dict.update(other_dict)  # This will result in the error message you mentioned.
my_dict

{'a': 1, 'b': 2, 'c': 3, 'd': 4}

In [67]:
x = (1,2,3,4,5,6)
y, *x = x
z, *x = x
t, *x = x
k, *x = x
x, y, z, t, k

([5, 6], 1, 2, 3, 4)

In [77]:
def error_logger(file_name, 
                 url="",
                 msg_error=""):
    def decorator(target_func):
        def wrapper(*args, **kwargs):
            data, success = target_func(*args, **kwargs)
            print("Collectors from target func:", args, kwargs)
            #if kwargs.get("error_msg", False):
            #    msg_error = kwargs["msg_error"]
            #else:
            #    msg_error = "undefined-msg-error"
            #if kwargs.get("file_name", False):
            #    file_name = kwargs["file_name"]
            #else:
            #    file_name = "file.txt"
            #if kwargs.get("url", False):
            #    url = kwargs["open_mode"]
            #else:
            #    url = "url-not-available"
            if not success:
                with open(file_name, "a") as f:
                    if not msg_error.endswith("\n"):
                        msg_error_newl = msg_error + "\n"
                    else:
                        msg_error_newl = msg_error
                    f.write(";".join([url, msg_error_newl]))
                print("Error written")
            else:
                print("No errors")
            return data
        return wrapper
    return decorator


In [68]:
f_name = "file.txt"
url = "https://www.farodevigo.es/galicia/2023/09/02/bng-propone-pacto-agua-combatir-91610658.html"
force_success = False

@error_logger("file.txt", url=url, msg_error="gpt-api-call-error")
def test(x: [float, int], 
         force_success: bool=False) -> tuple[[float, int], bool]:
    return x + 1.0, force_success

test(2, force_success)

TypeError: error_logger() got an unexpected keyword argument 'url'

In [ ]:
def error_logger(file_name, 
                 url="",
                 msg_error=""):
    def decorator(target_func):
        def wrapper(*args, **kwargs):
            data, success = target_func(*args, **kwargs)
            print("Collectors from target func:", args, kwargs)
            if not success:
                with open(file_name, "a") as f:
                    if not msg_error.endswith("\n"):
                        msg_error_newl = msg_error + "\n"
                    else:
                        msg_error_newl = msg_error
                    f.write(";".join([url, msg_error_newl]))
                print("Error written")
            else:
                print("No errors")
            return data
        return wrapper
    return decorator


In [70]:
import multiprocessing
class FileManager():
    def __init__(self) -> None:
        self.files_map = {}
    def add_files(self, 
                  files: list, 
                  **kwargs
                  ):
        for file_name in files:
            self._add_file(file_name, 
                           **kwargs)
    
    def _add_file(self, 
                  file_name: str,
                  **kwargs
                  ):
        if kwargs.get("open_mode", False):
            open_mode = kwargs["open_mode"]
        else:
            open_mode = "a"
        
        self.files_map[file_name] = open(file_name, open_mode)
    def write_on_file(self, *args, **kwargs):
        self._write_on_file(*args, **kwargs)
    
    def _write_on_file(self, 
                      file_name: str,
                      data: list, 
                      lock: multiprocessing.Lock,
                      pid: str=""
                      ):
        #with lock:
            if len(data) > 0:
                self.files_map[file_name].writelines(";".join([str(x) for x in data] + [pid + "\n"]))
    def close_all_files(self):
        for file in self.files_map.values():
            file.close()

def error_logger(manager: FileManager,
                 file_name: str,
                 error_msg_data: list, 
                 lock: multiprocessing.Lock,
                 pid: str=""
                 ):
    def decorator(target_func):
        def wrapper(*args, **kwargs):
            data, success = target_func(*args, **kwargs)
            print("Collectors from target func:", args, kwargs)
            if not success:
                manager.write_on_file(file_name,
                                      error_msg_data,
                                      lock,
                                      pid,
                                      )
            return data
        return wrapper
    return decorator

f_name = "file.txt"
url = "https://www.farodevigo.es/galicia/2023/09/02/bng-propone-pacto-agua-combatir-91610658.html"
error_data = ["gpt-api-call-error", url]
force_success = False
file_manager = FileManager()
file_manager.add_files([f_name])

@error_logger(file_manager,  "file.txt", error_data, multiprocessing.Lock, "1")
def test(x: [float, int], 
         force_success: bool=False) -> tuple[[float, int], bool]:
    return x + 1.0, force_success

print(test(2, force_success))

file_manager.close_all_files()

Collectors from target func: (2, False) {}
3.0


In [58]:
import multiprocessing
import time

# Create a global lock
global_lock = multiprocessing.Lock()

def function1():
    with global_lock:
        print("Inside function1")
        # Simulate some work
        time.sleep(2)

def function2():
    with global_lock:
        print("Inside function2")
        # Simulate some work
        time.sleep(2)

# Create two separate processes to call the functions simultaneously
#if __name__ == "__main__":
process1 = multiprocessing.Process(target=function1)
process2 = multiprocessing.Process(target=function2)

process1.start()
process2.start()

In [71]:
import multiprocessing
import time

def my_function(lock):
    print("fd")
    with lock:
        print(f"Function started in process {multiprocessing.current_process().name}")
        time.sleep(3)  # Simulate some work
        print(f"Function finished in process {multiprocessing.current_process().name}")

#if __name__ == "__main__":
lock = multiprocessing.Lock()

# Create two separate processes, each running the same function
process1 = multiprocessing.Process(target=my_function, args=(lock,))
process2 = multiprocessing.Process(target=my_function, args=(lock,))

process1.start()
process2.start()




In [72]:
process1.join()
process2.join()

In [5]:
with open("h", "a") as file:
    file.write("hola\n")

In [11]:
url1 = "https://www.elmundo.es/opinion/columnistas/2023/07/04/64a45a85e9cf4a65458b457f.html#ancladecomentarios"
url2 = "https://www.elmundo.es/opinion/columnistas/2023/07/04/64a45a85e9cf4a65458b457f.html"

# pad strings if needed
n1 = len(url1)
n2 = len(url2)
if n1 > n2:
    url2 = url2 + " " * abs(n1 - n2)
elif n1 < n2:
    url1 = url1 + " " * abs(n1 - n2)

len(url1), \
len(url2)

(102, 102)

Parse string to array

In [18]:
url1_arr = np.array(list(url1))
url2_arr = np.array(list(url2))
pairwise_equal = (url1_arr == url2_arr).astype(int)
pairwise_equal

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Try to shuffle second array to see how dissimilarity value rises to almost one

In [21]:
url1_arr = np.array(list(url1))
url2_arr = np.array(list(url2))
np.random.shuffle(url2_arr)
pairwise_equal = (url1_arr == url2_arr).astype(int)
print(pairwise_equal)

dissimilarity = jaccard(np.ones_like(pairwise_equal), pairwise_equal)
# The greater, the more dissimilar
dissimilarity

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]


0.9509803921568627

## HTML Character entities

In [23]:
import html

entity = "&apos;"
decoded = html.unescape(entity)
print(decoded)

'


In [25]:
text = "El turista Ivan Dimitrov , de 27 años, que vive en Bristol (Reino Unido) con su novia, Hayley Bracey, y que escribió el nombre de ambos en una de las paredes del Coliseo de Roma trató de disculparse en una carta enviada a la Fiscalía de Roma, al alcalde, Roberto Gualtieri, y al ayuntamiento de la capital italiana en la que asegura que desconocía la antigüedad del monumento. &quot;Reconozco con total vergüenza que sólo después de lo sucedido me enteré de la antigüedad del monumento&quot;, escribió el joven de origen búlgaro en la carta que hoy publica el diario romano &quot;Il Messaggero&quot;. Los dos estaban visitando la capital italiana durante un viaje de tres semanas por Europa y durante una visita al Coliseo, el hombre escribió la frase &quot;Ivan+Hayley 23&quot; en una pared, como se puede ver en un vídeo colgado por otro turista en una plataforma y que se hizo viral. El ministro de Cultura italiano, Gennaro Sangiuliano, prometió entonces que se perseguiría este acto de vandalismo y después de que los Carabineros rastrearan las imágenes de las cámaras de seguridad y los registros de los hoteles y se consiguió identificar al turista mientras estaban en Bulgaria. Sangiuliano aseguró que en caso de denuncia, el ministerio de Cultura se presentará como parte civil. Se desconoce si se ha abierto algún procedimiento judicial al joven. Según el diario, el joven inmediatamente se disculpó por su gesto y dijo a los Carabineros que estaba muy arrepentido y también preocupado por las consecuencias de su acto, pues se le explicó que se arriesga a una pena de prisión de dos a cinco años y una multa de entre 2.500 y 15.000 euros. &quot;Consciente de la gravedad del acto cometido, con estas líneas quisiera dirigir mis más sinceras y sinceras disculpas a los italianos y al mundo entero por los daños causados a un bien que, de hecho, es patrimonio de toda la humanidad&quot;, escribió en la carta. Sobre todo se disculpa con el ayuntamiento de Roma que &quot;con dedicación, cuidado y sacrificio custodian el inestimable valor histórico y artístico del Coliseo&quot; y luego intenta justificarse añadiendo que &quot;sólo más tarde se enteró de la antigüedad del monumento&quot;. Esta no es la primera vez que los visitantes del enorme anfiteatro deciden vandalizar sus muros. El pasado verano una turista canadiense fue sorprendida por los agentes de seguridad del Coliseo mientras escribía su nombre con una piedra en los mismos muros. En esa ocasión, los vigilantes pudieron alertar rápidamente a la policía, que acudió al lugar y denunció a la visitante."
print(text, "\n")
decoded = html.unescape(text)
print(decoded)

El turista Ivan Dimitrov , de 27 años, que vive en Bristol (Reino Unido) con su novia, Hayley Bracey, y que escribió el nombre de ambos en una de las paredes del Coliseo de Roma trató de disculparse en una carta enviada a la Fiscalía de Roma, al alcalde, Roberto Gualtieri, y al ayuntamiento de la capital italiana en la que asegura que desconocía la antigüedad del monumento. &quot;Reconozco con total vergüenza que sólo después de lo sucedido me enteré de la antigüedad del monumento&quot;, escribió el joven de origen búlgaro en la carta que hoy publica el diario romano &quot;Il Messaggero&quot;. Los dos estaban visitando la capital italiana durante un viaje de tres semanas por Europa y durante una visita al Coliseo, el hombre escribió la frase &quot;Ivan+Hayley 23&quot; en una pared, como se puede ver en un vídeo colgado por otro turista en una plataforma y que se hizo viral. El ministro de Cultura italiano, Gennaro Sangiuliano, prometió entonces que se perseguiría este acto de vandalism

In [8]:

hiperlink = "https://elpais.com/sociedad/2023-07-03/el-gobierno-aprobara-manana-el-fin-de-las-mascarillas-obligatorias-en-centros-sanitarios-farmacias-y-residencias.html#?rel=lom"
search_spec_char = re.search("[?=&+%@#]{1}", hiperlink)
s, e = search_spec_char.span()
hiperlink[:s]

'https://elpais.com/sociedad/2023-07-03/el-gobierno-aprobara-manana-el-fin-de-las-mascarillas-obligatorias-en-centros-sanitarios-farmacias-y-residencias.html'

In [25]:
def has_http_attribute_value(tag):
    for attr in tag.attrs:
        attr_value = tag.attrs[attr]
        if isinstance(attr_value, str) and ('http://' in attr_value or 'https://' in attr_value or re.search(r"(?:\/).+(?:\/).+(?:\/?)", attr_value)) :
            return True
    return False

In [38]:
url = "https://www.elconfidencial.com/tecnologia/2023-07-04/cifrado-reglamento-europeo-abuso-menores-whatsapp-mensajeria_3684963/"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=HEADERS)
response

SyntaxError: EOL while scanning string literal (2103130488.py, line 1)

In [34]:
parsed_hmtl = BeautifulSoup(response.content, "html.parser")
tags_with_urls = parsed_hmtl.find_all(lambda tag: has_http_attribute_value)
hiperlinks = []
for tags_with_url in tags_with_urls:
    for attr in tags_with_url.attrs:
        attr_value = tags_with_url.attrs[attr]
        if isinstance(attr_value, str) and ('http://' in attr_value or 'https://' in attr_value):
            hiperlink = tags_with_url[attr]
            if hiperlink.startswith("https://www.abc.es/"):
                hiperlinks.append(hiperlink)

In [35]:
hiperlinks

['https://www.abc.es/',
 'https://www.abc.es/rss/feeds/abc_portada.xml',
 'https://www.abc.es/rss/feeds/abc_ultima.xml',
 'https://www.abc.es/',
 'https://www.abc.es/',
 'https://www.abc.es/espana/elecciones-directo-pedro-sanchez-feijoo-yolanda-diaz-abascal-ultimas-noticias-generales-hoy-20230704084244-di.html',
 'https://www.abc.es/internacional/guerra-ucrania-rusia-directo-putin-zelenski-contraofensiva-ucraniana-20230705155105-di.html',
 'https://www.abc.es/espana/inmunidad-puigdemont-directo-ultima-hora-decision-justicia-ponsati-comin-hoy-20230705113122-di.html',
 'https://www.abc.es/gente/gonzalo-vargas-llosa-ultima-hora-sobre-estado-20230705121808-vi.html',
 'https://www.abc.es/play/television/noticias/audiencias-programa-ana-rosa-ganador-feijoo-sanchez-20230705120006-nt.html',
 'https://www.abc.es/recreo/entradas-taylor-swift-madrid-llegan-correos-codigo-20230705150833-nt.html',
 'https://www.abc.es/motor/dgt-confirma-situaciones-radares-pueden-multar-pese-20230705123454-nt.html'

In [64]:
url = "https://www.elconfidencial.com/espana/2022-10-12/feijoo-eliminara-ministerios-auditoria-moncloa_3505199/"
response = requests.get(url, headers=HEADERS)
parsed_hmtl = BeautifulSoup(response.content, "html.parser")
#links = [t.attrs["href"] for t in parsed_hmtl.find_all("a", href=re.compile("https?:.*"))]
p_tags = parsed_hmtl.find_all("p")
[p.text for p in p_tags]

[' El líder del PP, Alberto Núñez Feijóo, ha asegurado que las tres primeras cosas que haría de llegar al Palacio de la Moncloa sería suprimir entre seis y ocho ministerios, encargar una auditoría en las cuentas públicas y tomar en los primeros cien días "tres o cuatro decisiones estructurales". ',
 ' El líder de la oposición ha asumido estos compromisos en un desayuno informativo de El Mundo, en el que ha recalcado que el PP no renuncia a su objetivo de gobernar en solitario, algo que a su juicio lograría hacer con "pactos puntuales" si se hubiesen celebrado elecciones "anteayer". ',
 ' "Si no lo creyera no estaría tan animado como estoy. Mi partido en seis meses ha subido 10 puntos", ha señalado Feijóo al ser preguntado si prefería pactar con Vox o con el PNV. ',
 ' Por otra parte, Feijóo ha acusado al presidente del Gobierno, Pedro Sánchez, y al PSOE de "blindar al independentismo en Cataluña" y ha calificado de "sonrojante" y "humillante" que el president Pere Aragonès dijese "grac

In [61]:
search_spec_char = re.search("[=&+%@#?]{1}", "https://elpais.com/espana/2023-07-05/la-justicia-europea-retira-la-inmunidad-parlamentaria-de-carles-puigdemont.html?autoplay=1")
query_start = search_spec_char.span()[0]
hiperlink = "https://elpais.com/espana/2023-07-05/la-justicia-europea-retira-la-inmunidad-parlamentaria-de-carles-puigdemont.html?autoplay=1"[:query_start]

In [88]:
hiperlink

'https://elpais.com/espana/2023-07-05/la-justicia-europea-retira-la-inmunidad-parlamentaria-de-carles-puigdemont.html'

In [89]:
def search_articles(tag):
    return tag.name == "p" and not tag.attrs

In [92]:
url = "https://www.xataka.com/ecologia-y-naturaleza/ayer-fue-dia-caluroso-a-nivel-mundial-que-hay-registros-anterior-record-fue-anteayer"

response = requests.get(url)
parsed_hmtl = BeautifulSoup(response.content, "html.parser")

p_tags = parsed_hmtl.find_all(search_articles)
print([p.get_text() for p in p_tags])

['\n\nSuscribir\n', 'En 2016, me tocó escribir un artículo triste. Por primera vez en la historia de la humanidad, el dióxido de carbono en el aire superó las 400 partes por millón. Esa era una de las líneas rojas que tomábamos con referencia en la lucha contra el cambio climático. No era la primera que cruzábamos, pero como dice el profesor Matt England tenía un peso simbólico importante.', 'Hoy hemos cruzado otra: la temperatura media del mundo, por primera vez, ha superado los 17 grados. Eso fue el lunes 3 de julio. El 4 la temperatura subió 0,17 grados más.', "¿Líneas rojas? Sí, los esfuerzos globales por contener al cambio climático usan ciertos límites como referencia e inspiración: es una forma de medir la progresión de ese fenómeno global, sí; pero también es una forma de ponernos objetivos 'concretos'. Aunque luego no seamos capaces de conseguirlos.", 'Por eso mismo, estas líneas rojas tienen una parte importante de convención; es decir, no dejan de ser cifras elegidas con una

In [94]:
url = "https://www.xataka.com/ecologia-y-naturaleza/ayer-fue-dia-caluroso-a-nivel-mundial-que-hay-registros-anterior-record-fue-anteayer"

response = requests.get(url)

In [95]:
response.ti

<Response [200]>

In [96]:
import json

In [102]:
with open("./data/extraction checkpoint.json", "r") as file:
    data = json.load(file)

In [103]:
data

{'last_media_url': 'https://www.actualarganda.com'}

In [101]:
data["last_media_url"]

'https://www.dream-alcala.com'

## Noticias a través de p tags

In [6]:
def search_p_tags_noattrs(tag):
    return (tag.name == "p" and not tag.attrs) or ("h" in tag.name and not tag.attrs)

url1 = "https://www.xataka.com/ecologia-y-naturaleza/ayer-fue-dia-caluroso-a-nivel-mundial-que-hay-registros-anterior-record-fue-anteayer"
url2 = "https://okdiario.com/espana/feijoo-no-si-sera-hombre-o-mujer-pero-ya-digo-que-fiscal-general-del-estado-no-sera-del-pp-11211283"
url3 = "https://www.informacion.es/nacional/2023/09/03/gobierno-expedientara-directores-generales-aragon-91646243.html"
url4 = "https://www.abc.es/gente/maria-teresa-campos-ingresada-urgencia-hospital-madrid-20230903142634-nt.html"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
response = requests.get(url4, headers=headers)

### Find p or h tags, no attributes, extract text and clean new lines

In [7]:
parsed_html = BeautifulSoup(response.content, "html.parser")

p_tags = parsed_html.find_all(search_p_tags_noattrs)
text_clean_from_p_tags = [re.sub("\n+", "\n", p.get_text()) for p in p_tags]
text_clean_from_p_tags[:2]

['\n \nMaría Teresa Campos, ingresada de urgencia en un hospital de Madrid\n \n']

Other approach

In [8]:
from pprint import pprint

In [10]:
tags_with_text = parsed_html.html.body.find_all(lambda tag: tag.name in ("p", "h1", "h2"))
clean_paragraphs_str = "".join([re.sub("\n+", "\n", tag.get_text()) for tag in tags_with_text])
clean_paragraphs_list = [tag.get_text() for tag in tags_with_text]
parag_texts = str({i: x if x else "\n" for i, x in enumerate(clean_paragraphs_list)})[1:-1]
pprint(clean_paragraphs_str)

('Copiar enlaceFacebookTwitterWhatsappEmailEsta funcionalidad es sólo para '
 'registradosInicioMi perfilMis suscripcionesMis interesesNewsletters y '
 'alertasClub ABC PremiumNoticias guardadasAutores seguidosMi archivoConcursos '
 'y eventosContactoCerrar sesiónMaría Teresa Campos, ingresada de urgencia en '
 'un hospital de MadridLa popular presentadora ha ingresado en la Fundación '
 'Jiménez Díaz con pronóstico reservado. A lo largo de la tarde, el propio '
 'centro hospitalario emitirá un comunicadoTerelu Campos, devastada al hablar '
 'de su madre: «No soporto más verla sufrir»\n'
 '        ABC\n'
 'Esta funcionalidad es sólo para registradosCopiar '
 'enlaceFacebookTwitterWhatsappEmailEsta funcionalidad es sólo para '
 'registradosEsta funcionalidad es sólo para suscriptoresMaría Teresa Campos '
 'ha ingresado esta mañana en la Fundación Jiménez Díaz de Madrid con '
 "pronóstico reservado. Una noticia avanzada por el programa 'Fiesta de "
 "verano' y que este periódico ha podid

In [12]:
tags_with_text = parsed_html.find_all(lambda tag: tag.name in ("p", "h1", "h2"))
text_clean_from_tags = "".join([re.sub("\n+", "\n", tag.get_text()) for tag in tags_with_text])
len(text_clean_from_tags), text_clean_from_tags

(3756,
 "Copiar enlaceFacebookTwitterWhatsappEmailEsta funcionalidad es sólo para registradosInicioMi perfilMis suscripcionesMis interesesNewsletters y alertasClub ABC PremiumNoticias guardadasAutores seguidosMi archivoConcursos y eventosContactoCerrar sesiónMaría Teresa Campos, ingresada de urgencia en un hospital de MadridLa popular presentadora ha ingresado en la Fundación Jiménez Díaz con pronóstico reservado. A lo largo de la tarde, el propio centro hospitalario emitirá un comunicadoTerelu Campos, devastada al hablar de su madre: «No soporto más verla sufrir»\n        ABC\nEsta funcionalidad es sólo para registradosCopiar enlaceFacebookTwitterWhatsappEmailEsta funcionalidad es sólo para registradosEsta funcionalidad es sólo para suscriptoresMaría Teresa Campos ha ingresado esta mañana en la Fundación Jiménez Díaz de Madrid con pronóstico reservado. Una noticia avanzada por el programa 'Fiesta de verano' y que este periódico ha podido confirmar. Según ha podido saber ABC, durante l

In [13]:
tags_with_text = parsed_html.find_all(lambda tag: tag.name in ("p", "h1"))
text_clean_from_tags = "".join([re.sub("\n+", "\n", tag.get_text()) for tag in tags_with_text])
len(text_clean_from_tags), text_clean_from_tags

(3512,
 "Copiar enlaceFacebookTwitterWhatsappEmailEsta funcionalidad es sólo para registradosInicioMi perfilMis suscripcionesMis interesesNewsletters y alertasClub ABC PremiumNoticias guardadasAutores seguidosMi archivoConcursos y eventosContactoCerrar sesiónMaría Teresa Campos, ingresada de urgencia en un hospital de Madrid\n        ABC\nEsta funcionalidad es sólo para registradosCopiar enlaceFacebookTwitterWhatsappEmailEsta funcionalidad es sólo para registradosEsta funcionalidad es sólo para suscriptoresMaría Teresa Campos ha ingresado esta mañana en la Fundación Jiménez Díaz de Madrid con pronóstico reservado. Una noticia avanzada por el programa 'Fiesta de verano' y que este periódico ha podido confirmar. Según ha podido saber ABC, durante las próximas horas, será el propio centro hospitalario el que emita un comunicado para aclarar cómo se encuentra la periodista. Pese a que por el momento se desconocen más datos, el estado de salud de la presentadora preocupa a círculo más cerca

In [14]:
tags_with_text = parsed_html.find_all(lambda tag: tag.name in ("p"))
text_clean_from_tags = "".join([re.sub("\n+", "\n", tag.get_text()) for tag in tags_with_text])
len(text_clean_from_tags), text_clean_from_tags

(3445,
 "Copiar enlaceFacebookTwitterWhatsappEmailEsta funcionalidad es sólo para registradosInicioMi perfilMis suscripcionesMis interesesNewsletters y alertasClub ABC PremiumNoticias guardadasAutores seguidosMi archivoConcursos y eventosContactoCerrar sesión\n        ABC\nEsta funcionalidad es sólo para registradosCopiar enlaceFacebookTwitterWhatsappEmailEsta funcionalidad es sólo para registradosEsta funcionalidad es sólo para suscriptoresMaría Teresa Campos ha ingresado esta mañana en la Fundación Jiménez Díaz de Madrid con pronóstico reservado. Una noticia avanzada por el programa 'Fiesta de verano' y que este periódico ha podido confirmar. Según ha podido saber ABC, durante las próximas horas, será el propio centro hospitalario el que emita un comunicado para aclarar cómo se encuentra la periodista. Pese a que por el momento se desconocen más datos, el estado de salud de la presentadora preocupa a círculo más cercano, quienes en las últimas semanas no han escondido su inquietud.En

In [15]:
article_summ = " María Teresa Campos has been admitted to the Jiménez Díaz Foundation in Madrid with a reserved prognosis. The health status of the presenter is a concern to her closest circle. Terelu Campos, her daughter, talked about her mother's illness and expressed her fear and sadness. Carmen Borrego, another daughter, mentioned that their priority is their mother's well-being. María Teresa Campos is suffering from an important cognitive deterioration, and her daughters are trying to protect her from negative news, such as the recent death of Hilario Pino."
len(article_summ)

552

In [28]:
n_ls = 1
for x in text_clean_from_p_tags:
    print(x, end="\n"*n_ls)


Apología del franquismo | El Gobierno expedientará a dos directores generales de Aragón por exaltar el franquismo

Síguenos en redes sociales:


Noticia guardada en tu perfil

La oposición pide el cese de la directora general de Justicia del Gobierno de Aragón por "apología del franquismo"
M. C. L.

La oposición pide el cese de la directora general de Justicia del Gobierno de Aragón por "apología del franquismo"
Compartir el artículo


### Extract all the text and clean new lines

In [29]:
text_clean_from_full_html = re.sub("\n+", "\n", parsed_hmtl.get_text())
text_clean_from_full_html[:50]

'\nApología del franquismo | El Gobierno expedientar'

In [30]:
print(text_clean_from_full_html)


Apología del franquismo | El Gobierno expedientará a dos directores generales de Aragón por exaltar el franquismo
                DANA
            
                Salud mental
            
                Cantera
            
                Denuncia pareja Alicante
            
                Bici
            
                Vuelta Ciclista en Alicante
            
                Noticias de Alicante
            
                 Alicante
             
                        La Crónica de L’Alacantí
            
                        San Vicente
            
                        Sant Joan
            
                        El Campello
            
                        Sucesos
            
                        Hércules CF
            
                        Hogueras
            
                        Barrios
            
                        Vídeos
            
                        Fotos
            
                 Elche
             
                     

In [31]:
print(f"Number of characters from p tags: {len(''.join(text_clean_from_p_tags))}")
print(f"Number of characters from full text: {len(text_clean_from_full_html)}")

Number of characters from p tags: 432
Number of characters from full text: 12169


In [32]:
text_chatgpt = """
Title: Feijóo: "El Fiscal General del Estado no será del PP"

Body:
En una entrevista con OKDIARIO, Alberto Núñez Feijóo, líder del Partido Popular y candidato a la presidencia, afirmó que el Fiscal General del Estado no será miembro del PP. Feijóo señaló que no nombrará a alguien que sea parte del Comité Ejecutivo de su partido o a alguien recién designado en otro órgano. Hizo referencia a experiencias previas en las que se criticó el nombramiento de un miembro del partido como Fiscal General o como magistrado del Tribunal Constitucional.

Feijóo también abordó otros temas en la entrevista, como la situación en el Ministerio de Justicia y la necesidad de contar con profesionales competentes en el sistema judicial. Además, mencionó su intención de recuperar los delitos de sedición y malversación en su versión anterior en los primeros 100 días de su mandato, si los españoles le otorgan el mandato.

Tags: Alberto Núñez Feijóo, Fiscal General del Estado, Partido Popular, Ministerio de Justicia, nombramientos, delito de sedición, delito de malversación.
"""

In [11]:
print(f"Number of characters from chatgpt: {len(text_chatgpt)}")

Number of characters from chatgpt: 1067


In [12]:
chars_in_token = [2, 3, 4]
for chars in chars_in_token:
    print(f"Case token is {chars} characters")
    print(f"\tNumber of tokens from p tags: {len(''.join(text_clean_from_p_tags)) // chars}")
    print(f"\tNumber of tokens from full text: {len(text_clean_from_full_html) // chars}")
    print(f"\tNumber of tokens from chatgpt: {len(text_chatgpt) // chars}", end="\n"*2)

Case token is 2 characters
	Number of tokens from p tags: 2289
	Number of tokens from full text: 3161
	Number of tokens from chatgpt: 533

Case token is 3 characters
	Number of tokens from p tags: 1526
	Number of tokens from full text: 2107
	Number of tokens from chatgpt: 355

Case token is 4 characters
	Number of tokens from p tags: 1144
	Number of tokens from full text: 1580
	Number of tokens from chatgpt: 266



### Using openai to extract keys

In [3]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [33]:
#prompt_role_complex = """Your commitment is to review text from news, which contain the headline and body content. However, there might be other pieces of text irrelevant which are irrelevant, like footer references to images related to the news, social media platforms footers, subscription texts, users' comments or texts related to comments, among others. Therefore, you have to be able to extract the content / body, the headline / title of the news and, the tags related to the news, if possible , while avoiding irrelevant text. The headline / title and content / body of the news should keep the core and details of the information contained, but you must keep a tone of independence of opinion like a journalist. Do not notify at the output about any other matters, like what you recommend or what you removed from the original text. This is the text: \n"""
#prompt_role_simple = "You receive news. Extract its title, the main content, tags (if possible), creation date (if possible) and update date (if possible). This is the news: "
with open("../data/role-prompt-body-extraction.txt", "r") as file:
    prompt_role_complex = file.read()
prompt_order_full = text_clean_from_full_html
#prompt_order_p_tags = '\n'.join(text_clean_from_p_tags)
#total_prompt = prompt_role + prompt_order
#total_prompt
prompt_order_full

'\nApología del franquismo | El Gobierno expedientará a dos directores generales de Aragón por exaltar el franquismo\n                DANA\n            \n                Salud mental\n            \n                Cantera\n            \n                Denuncia pareja Alicante\n            \n                Bici\n            \n                Vuelta Ciclista en Alicante\n            \n                Noticias de Alicante\n            \n                 Alicante\n             \n                        La Crónica de L’Alacantí\n            \n                        San Vicente\n            \n                        Sant Joan\n            \n                        El Campello\n            \n                        Sucesos\n            \n                        Hércules CF\n            \n                        Hogueras\n            \n                        Barrios\n            \n                        Vídeos\n            \n                        Fotos\n            \n                 El

In [34]:
prompt_role_complex

'You are a journalist who adheres to the following approaches:\n\n1. Write a summary from the article body in third-person point of view.\n2. The summary must be written in the language of the article body.\n3. Include important details.\n\nPlease extract and output the following information from the provided text:\n\nTokens: [tokens]\nBody Summary: [body summary]\n\nReplace \'[number tokens]\' with the number of tokens in the incoming text and replace \'[body summary]\' \nwith the summary of the article body. If the information could not be found, then write "BodyNotFound" neither \nwith quotes nor brackets. Prompted paragraphs are mixed with other garbage paragraphs. Paragraphs that belong to the body \ncould appear aranged sequentially. Join the paragraphs and don\'t output the enumeration. \nDon\'t explain anything regarding the outputted text. Please stand by for me to provide the text.'

In [35]:
print(prompt_order_full)


Apología del franquismo | El Gobierno expedientará a dos directores generales de Aragón por exaltar el franquismo
                DANA
            
                Salud mental
            
                Cantera
            
                Denuncia pareja Alicante
            
                Bici
            
                Vuelta Ciclista en Alicante
            
                Noticias de Alicante
            
                 Alicante
             
                        La Crónica de L’Alacantí
            
                        San Vicente
            
                        Sant Joan
            
                        El Campello
            
                        Sucesos
            
                        Hércules CF
            
                        Hogueras
            
                        Barrios
            
                        Vídeos
            
                        Fotos
            
                 Elche
             
                     

In [36]:
openai_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": prompt_role_complex},
        {"role": "user", "content": prompt_order_full},
    ]
)
message_content = openai_response["choices"][0].message["content"]
print(message_content)

Tokens: 231
Body Summary: El Ministerio de la Presidencia, a través de la Secretaría de Estado de Memoria Democrática, ha anunciado el inicio de actuaciones previas a dos expedientes sancionadores a sendos directores generales del Gobierno de Aragón por exaltar el franquismo. Se trata, concretamente, de la directora general de Justicia, Esmeralda Pastor Estrada, quien exhibió en sus redes sociales la bandera preconstitucional y colgó mensajes de apoyo a la dictadura de Franco, publicaciones que han sido borradas. En cumplimiento de la Ley de Memoria Democrática, el Gobierno actuará de la misma manera con el director general de Caza y Pesca del Gobierno de Aragón, Jorge Valero, por las manifestaciones vertidas en Facebook sobre José Millán Astray, al que felicitó en el 139 aniversario de su nacimiento. En otros mensajes, Valero se hizo eco de una manifestación en defensa del citado fundador de la Legión, y afirmó que "Franco evitó que los rojos siguieran violando y matando monjas". Amba

In [24]:
len(message_content.split("\n"))

5

In [25]:
data = [x.split(": ")[1].strip() for x in message_content.split("\n")]
keys = (("title", 0), ("body", 4), ("creation_date", 2), ("update_date", 3), ("tags", 1))
data = {key: data[idx] for (key, idx)  in keys}
data

{'title': 'Feijóo',
 'body': 'En una entrevista, Alberto Núñez Feijóo, líder del Partido Popular (PP) en España, afirmó que el próximo Fiscal General del Estado no será del PP. Feijóo destacó que no nombrará a un miembro de su partido para ese cargo y criticó la elección de anteriores fiscales generales con afiliación política. También mencionó la intención de aumentar el número de jueces y magistrados y la necesidad de reformar la Administración de Justicia. Además, habló sobre la recuperación de los delitos de sedición y malversación.',
 'creation_date': '06/07/2023 06:58',
 'update_date': '06/07/2023 08:57',
 'tags': 'Fiscal General del Estado, PP, Justicia'}

#### OpenAI wih babbage-002

In [4]:
text = """
 En una operación conjunta entre la Guardia Civil y la Policía Húngara, tres delincuentes de alto nivel, 
 buscados en Hungría, han sido detenidos en Calpe, al norte de Alicante, en España. Estos ciudadanos 
 húngaros estaban en la lista de los delincuentes más buscados de Europa. El grupo criminal estaba 
 involucrado en el cultivo y procesamiento de plantas de Cannabis sativa en Hungría. La plantación fue 
 desmantelada y se incautaron 165 kilogramos de marihuana y cocaína, así como activos valorados en 
 millones de florines húngaros. Dos miembros de la organización fueron detenidos en Hungría anteriormente, 
 mientras que los otros tres fugitivos fueron detenidos en España. Sin embargo, todavía hay dos miembros
 de la organización que siguen huidos. Las autoridades solicitan información sobre ellos y podrían 
 enfrentarse a cadena perpetua si se demuestra su culpabilidad.
"""

In [15]:
openai_response = openai.Completion.create(
    model="babbage-002",
    #prompt="Make up a news article"#"Make a summary of less than 50 words. " + text
    prompt="Make a summary of more than 50 words. " + text
)
#message_content = openai_response["choices"][0].message["content"]
message_content = openai_response
print(message_content)

{
  "id": "cmpl-7ue19NSFOcXV6qv7LULCUQnoXu3mp",
  "object": "text_completion",
  "created": 1693734551,
  "model": "babbage-002",
  "choices": [
    {
      "text": " \n \n Mientras tanto, tres vietnamitas fueron arrestados por su presunta",
      "index": 0,
      "logprobs": null,
      "finish_reason": "length"
    }
  ],
  "usage": {
    "prompt_tokens": 249,
    "completion_tokens": 16,
    "total_tokens": 265
  }
}


### Only body summary

In [6]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [10]:
prompt_role_summary_only = """
You are a journalist that respect these approaches:
1. Write the summary in third-person point of view.
2. Write the summary into the language of incoming text.
3. Add important details.

You have to extract and output the next information from news articles:

Tokens: [tokens]
Body Summary: [body summary]

Replace [tokens] with the number of tokens of the incoming text and [body summary] 
with the extracted information from the provided text. Stand by for the article:
"""

In [11]:
body = """
España mira al cielo con cara de no entender. Junio, tradicionalmente el primero de los meses secos, ha sido el más lluvioso del año en ciudades como Málaga o Huesca. En Madrid, incluso, deja una tormenta de granizo. Pero no es suficiente. Las precipitaciones anuales continúan por debajo de los valores normales. Han sido demasiado cortas e irregulares, y los embalses siguen sin recuperarse. Estas infraestructuras han visto menguar su volumen de agua almacenada y se encuentran al 47,4% de su capacidad, una décima menos que la semana anterior. Son datos inferiores a los de las mismas fechas de 2022 y se sitúan 19 puntos por debajo del valor medio de la última década (66,7%). La situación preocupa. Por su parte, la cantidad de agua embalsada en las presas destinadas a la producción de energía eléctrica también ha menguado. Su reserva de agua se encuentra 7 décimas por debajo con respecto a la última semana, de forma que estos embalses se encuentran al 64,3% de su capacidad, igualmente por debajo de los valores medios de la última década (77%). Una cosa es clara: será preciso continuar con la pertinaz lucha que nuestro país mantiene desde toda la vida contra la sequía. En este panorama, surgen algunas preguntas: ¿nuestras reservas de agua están garantizadas? ¿Está asegurada el agua de uso humano, los caudales ecológicos o la producción de energía? ¿Qué hacen las centrales hidroeléctricas en épocas de sequía? Santiago Domínguez, responsable de generación hidroeléctrica de Endesa en España, tiene claro que este año está siendo malo. “El año pasado fue seco, por debajo de la media, y este año está siendo peor”. Arrastramos, dice, una situación de estrés hídrico que se ha ido encadenando. Para el trabajador de Endesa, que tiene centrales distribuidas por casi todo el mapa, desde Galicia a Cataluña, Extremadura o Andalucía, la situación de escasez no es nueva en el sur. Pero este año es “enorme” y ahora mismo hay una “situación de estrés hídrico” en las 3 zonas en las que operan. “Nuestras reservas se encuentran por debajo de la media nacional y están próximas al 30% con respecto a la capacidad total”, apunta, sin paliativos. “Una de las mayores, la de Mequinenza, en el Ebro, nunca ha estado en niveles tan bajos”. De hecho, tres de ellas (una en la cuenca del Ebro y dos en la cuenca del Guadiana) ya se encuentran paradas por falta de agua disponible para poder turbinar y generar electricidad. “El año pasado teníamos la previsión de que tendríamos que parar hacia septiembre-octubre, pero llegaron lluvias y no hubo que pararlas. Este año la previsión es que paren en agosto-septiembre, y en agosto es improbable que llueva…”, explica. Pero ¿por qué para una central hidroeléctrica? A grandes rasgos, por el bajo nivel de los embalses. “Cuando la entrada de agua no permite mantener la salida, la central se detiene para asegurar los caudales ecológicos, el abastecimiento humano y el riego”. En España, la ley establece una utilidad estricta para el uso del agua de los embalses: agua para consumo humano, agua para el campo y usos industriales, agua para generar energía y agua de uso recreativo. “El primero es el uso prioritario del agua, es decir, si peligra el consumo humano, todo lo que viene detrás se ajusta”, explica Domínguez. “Por eso el agua consuntiva, para el consumo humano, jamás se ha restringido (salvo excepciones muy particulares en las que la reserva es muy pequeña, no así en grandes embalses), pero sí se restringe el agua en otros sectores”, detalla. “En segunda posición se encuentra el agua para riego, cuyo consumo es muy relevante, muchísimo mayor que el humano y el hidroeléctrico, y este año ha sufrido ciertas restricciones importantes por falta de agua”, continúa. En tercer lugar se encuentra el uso hidroeléctrico, que no consume el agua. “La toma de un tramo del río, la turbina y la devuelve un poco más abajo, aunque es cierto que en ese tramo que une ambas partes el agua no está en el cauce, pero es un agua que como tal no se consume”, constata Domínguez. Para remediarlo, las Confederaciones Hidrográficas establecen una serie de medidas, como los caudales ecológicos, ya que la preservación de la biodiversidad de estas instalaciones es una obligación legal, incluso en situaciones de sequía. “Siempre se deben respetar los caudales ecológicos, por preservar el valor ecológico de la biodiversidad en nuestros ríos. Por lo que un río nunca se puede secar. El caudal mínimo debe respetarse antes del riego y después del consumo humano. El incumplimiento puede tener consecuencias penales. Si cualquier agente no lo respetase se expondría a una condena legal”, afirma. “La hidroeléctrica está en tercera posición, así que gestionamos nuestras instalaciones enfocados a asegurar los otros usos -continúa-. En los embalses donde hay reserva de agua suficiente, se desembalsa al ritmo que fijan las Confederaciones Hidrográficas para que en el río haya caudal suficiente como para atender todos los fines”. ¿Quién gestiona los embalses?Aunque pueda pensarse lo contrario, las eléctricas no deciden libremente cuándo utilizar el agua de un embalse para generar electricidad. Es la Administración del Estado quien gestiona las reservas de agua, a través de variedad de organismos regionales (Organismos de Cuenca) y, dentro de ellos, las Comisiones de Desembalse. El objetivo de estas comisiones es coordinar la variedad de necesidades de todos los beneficiarios del agua para garantizar el cumplimiento de sus usos prioritarios. “Por ejemplo, en el caso del río Ebro, hay embalses que deben garantizar el abastecimiento de Zaragoza, el riego de los campos, el agua para las industrias que la necesitan y las hidroeléctricas para generar electricidad”, ejemplifica Domínguez. En estos casos, se organizan periódicamente Comisiones de Desembalse integradas por agentes de todos los sectores. “En ellas se deben poner de acuerdo y a veces es complicado porque no hay agua para atender todos los usos. En estos casos, cada agente recibe unas instrucciones: a los regantes les dicen, por ejemplo, que pueden usar sólo una parte del volumen autorizado en condiciones normales. Y lo mismo con el resto de usos. Todos están sometidos al mejor reparto posible del volumen de agua disponible”, comenta el responsable. En la actualidad, aunque una mayor producción hidroeléctrica contribuiría a mitigar el impacto del precio del gas y la crisis climática, al reducir la necesidad de generar energía mediante centrales térmicas, nos encontramos en un año en el que la aportación hidroeléctrica está muy limitada debido a la escasez de agua. Para ello, “contamos con otro tipo de centrales, como las fotovoltaicas, que este año producirán algo más que en años anteriores por tamaño y número de instalaciones”. Asimismo, las compañías tratan de reducir la necesidad del uso del gas cubriendo la demanda con energía hidroeléctrica fuera de las horas en las que las tecnologías renovables como la solar no están disponibles. Por ejemplo, potenciando la generación hidroeléctrica por la noche. ¿Y si nos enfrentáramos a una sequía que pusiera en riesgo los usos consuntivos del agua, qué pasaría con la producción hidroeléctrica? “La primera vía sería limitar su uso al máximo, empezando por la generación hidroeléctrica. Nos ajustarían el caudal que podemos tomar para evitar situaciones de estrés en estos tramos del río entre el punto de toma y el de restitución del agua al cauce o lo restringirían del todo”, argumenta. Después se limitaría el regadío, llegando “al extremo de cancelarse del todo”. Y, ya por último, se podrían modificar los caudales ecológicos establecidos por ley. Llegados a ese punto hipotético de catástrofe futurista, explica, habría que cerrar el río para que todos los embalses pudieran sostener el agua que abastece a las ciudades. Por fortuna, “estamos muy lejos de comprometer el abastecimiento y los caudales ecológicos mínimos que aseguran la biodiversidad y el suministro. Por mucha sequía que atravesemos, estaremos siempre lejos de comprometerlo”, aclara. “Las normas y mecanismos que regulan el agua de los embalses en España son claras y, en nuestro país, somos expertos en gestión del agua y en sus problemas adicionales, tanto en sequía como en inundaciones; estamos muy preparados”.
"""

In [12]:
body.split("España mira ")

['\n',
 'al cielo con cara de no entender. Junio, tradicionalmente el primero de los meses secos, ha sido el más lluvioso del año en ciudades como Málaga o Huesca. En Madrid, incluso, deja una tormenta de granizo. Pero no es suficiente. Las precipitaciones anuales continúan por debajo de los valores normales. Han sido demasiado cortas e irregulares, y los embalses siguen sin recuperarse. Estas infraestructuras han visto menguar su volumen de agua almacenada y se encuentran al 47,4% de su capacidad, una décima menos que la semana anterior. Son datos inferiores a los de las mismas fechas de 2022 y se sitúan 19 puntos por debajo del valor medio de la última década (66,7%). La situación preocupa. Por su parte, la cantidad de agua embalsada en las presas destinadas a la producción de energía eléctrica también ha menguado. Su reserva de agua se encuentra 7 décimas por debajo con respecto a la última semana, de forma que estos embalses se encuentran al 64,3% de su capacidad, igualmente por de

In [13]:
openai_response2 = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": prompt_role_summary_only},
        {"role": "user", "content": body},
    ]
)
message_content2 = openai_response2["choices"][0].message["content"]
print(message_content2)

Tokens: 432

Body Summary: 
Spain is experiencing a rainy month of June, which is unusual for this time of year. However, despite the rainfall, the country's reservoirs are still below their normal levels. The water levels in reservoirs for electricity production have also decreased. Santiago Domínguez, a representative from Endesa, explains that the low levels of water in the reservoirs have led to the shutdown of some hydroelectric power plants. He further elaborates on the priority of water usage, with consumption for human use being the top priority, followed by irrigation and then hydroelectric power generation. Domínguez emphasizes the importance of maintaining ecological flow rates in rivers, even during periods of drought. The management of reservoirs in Spain is regulated by government agencies at both the national and regional levels. While hydroelectric power production has been limited due to water scarcity this year, other sources of renewable energy, such as solar power, 

In [17]:
message_content2.split("\n")

['Tokens: 432',
 '',
 'Body Summary: ',
 "Spain is experiencing a rainy month of June, which is unusual for this time of year. However, despite the rainfall, the country's reservoirs are still below their normal levels. The water levels in reservoirs for electricity production have also decreased. Santiago Domínguez, a representative from Endesa, explains that the low levels of water in the reservoirs have led to the shutdown of some hydroelectric power plants. He further elaborates on the priority of water usage, with consumption for human use being the top priority, followed by irrigation and then hydroelectric power generation. Domínguez emphasizes the importance of maintaining ecological flow rates in rivers, even during periods of drought. The management of reservoirs in Spain is regulated by government agencies at both the national and regional levels. While hydroelectric power production has been limited due to water scarcity this year, other sources of renewable energy, such as

In [19]:
len(message_content2.split(":"))

3

In [21]:
message_content2.split(": ")

['Tokens',
 '432\n\nBody Summary',
 "\nSpain is experiencing a rainy month of June, which is unusual for this time of year. However, despite the rainfall, the country's reservoirs are still below their normal levels. The water levels in reservoirs for electricity production have also decreased. Santiago Domínguez, a representative from Endesa, explains that the low levels of water in the reservoirs have led to the shutdown of some hydroelectric power plants. He further elaborates on the priority of water usage, with consumption for human use being the top priority, followed by irrigation and then hydroelectric power generation. Domínguez emphasizes the importance of maintaining ecological flow rates in rivers, even during periods of drought. The management of reservoirs in Spain is regulated by government agencies at both the national and regional levels. While hydroelectric power production has been limited due to water scarcity this year, other sources of renewable energy, such as so

In [14]:
if "Body Summary: " in message_content2:
    body_summary = message_content2.split("Body Summary: ")[-1]
else:
    body_summary = message_content2
body_summary

"\nSpain is experiencing a rainy month of June, which is unusual for this time of year. However, despite the rainfall, the country's reservoirs are still below their normal levels. The water levels in reservoirs for electricity production have also decreased. Santiago Domínguez, a representative from Endesa, explains that the low levels of water in the reservoirs have led to the shutdown of some hydroelectric power plants. He further elaborates on the priority of water usage, with consumption for human use being the top priority, followed by irrigation and then hydroelectric power generation. Domínguez emphasizes the importance of maintaining ecological flow rates in rivers, even during periods of drought. The management of reservoirs in Spain is regulated by government agencies at both the national and regional levels. While hydroelectric power production has been limited due to water scarcity this year, other sources of renewable energy, such as solar power, are being utilized. In th

In [32]:
message_content2

"Tokens: 432\n\nBody Summary: \nSpain is experiencing a rainy month of June, which is unusual for this time of year. However, despite the rainfall, the country's reservoirs are still below their normal levels. The water levels in reservoirs for electricity production have also decreased. Santiago Domínguez, a representative from Endesa, explains that the low levels of water in the reservoirs have led to the shutdown of some hydroelectric power plants. He further elaborates on the priority of water usage, with consumption for human use being the top priority, followed by irrigation and then hydroelectric power generation. Domínguez emphasizes the importance of maintaining ecological flow rates in rivers, even during periods of drought. The management of reservoirs in Spain is regulated by government agencies at both the national and regional levels. While hydroelectric power production has been limited due to water scarcity this year, other sources of renewable energy, such as solar pow

In [55]:
%timeit re.search("Tokens: (\d+).*Body Summary:.*", message_content2, flags=re.DOTALL).groups()
%timeit re.search("Tokens: (\d+).*Body Summary:", message_content2, flags=re.DOTALL).groups()
%timeit re.search("Tokens: (\d+).*", message_content2, flags=re.DOTALL).groups()

2.08 µs ± 10.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.07 µs ± 14.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
963 ns ± 2.39 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [59]:
re.compile("Tokens: (\d+).*", flags=re.DOTALL).search(message_content2).groups()[0]

'432'

In [61]:
%timeit re.search(".*Body Summary:(.*)", message_content2, flags=re.DOTALL).groups()
%timeit re.search("Body Summary:(.*)", message_content2, flags=re.DOTALL).groups()

2.14 µs ± 5.51 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
1.07 µs ± 5.02 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [67]:
re.search("Body Summary:(.*)", flags=re.DOTALL).groups()[0]

" \nSpain is experiencing a rainy month of June, which is unusual for this time of year. However, despite the rainfall, the country's reservoirs are still below their normal levels. The water levels in reservoirs for electricity production have also decreased. Santiago Domínguez, a representative from Endesa, explains that the low levels of water in the reservoirs have led to the shutdown of some hydroelectric power plants. He further elaborates on the priority of water usage, with consumption for human use being the top priority, followed by irrigation and then hydroelectric power generation. Domínguez emphasizes the importance of maintaining ecological flow rates in rivers, even during periods of drought. The management of reservoirs in Spain is regulated by government agencies at both the national and regional levels. While hydroelectric power production has been limited due to water scarcity this year, other sources of renewable energy, such as solar power, are being utilized. In t

### Extract html, body, div and all the text and clean new lines

In [38]:
def search_p_tags_noattrs(tag):
    return (tag.name == "p" and not tag.attrs) or ("h" in tag.name and not tag.attrs)

url1 = "https://www.xataka.com/ecologia-y-naturaleza/ayer-fue-dia-caluroso-a-nivel-mundial-que-hay-registros-anterior-record-fue-anteayer"
url2 = "https://okdiario.com/espana/feijoo-no-si-sera-hombre-o-mujer-pero-ya-digo-que-fiscal-general-del-estado-no-sera-del-pp-11211283"
url3 = "https://www.20minutos.es/noticia/5145201/0/asi-ha-sido-rescate-una-mujer-zaragoza-tras-las-fuertes-lluvias/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
response = requests.get(url3, headers=headers)

parsed_hmtl = BeautifulSoup(response.content, "html.parser")
divs_in_body = parsed_hmtl.html.body.find_all('div')

In [41]:
texts = [re.sub("\n+", "\n", x.get_text()) for x in divs_in_body]
print(texts)

['\n', '', '\n', '', '', 'ZaragozaEntrarRegistrarseMi PerfilSalir', 'ZaragozaEntrarRegistrarseMi PerfilSalir', '', '', 'Zaragoza', '', 'EntrarRegistrarse', 'Mi PerfilSalir', '', "\nDIRECTOArranca una campaña del 23-J marcada por el 'bibloquismo' y un voto por correo disparado\nÚLTIMA HORAEl juez envía a prisión provisional al asesino de Concha en Tirso de Molina y a su cómplice\n", '\n', '', "\nDIRECTOArranca una campaña del 23-J marcada por el 'bibloquismo' y un voto por correo disparado\nÚLTIMA HORAEl juez envía a prisión provisional al asesino de Concha en Tirso de Molina y a su cómplice\n", "\nDIRECTOArranca una campaña del 23-J marcada por el 'bibloquismo' y un voto por correo disparado\n", "\nDIRECTOArranca una campaña del 23-J marcada por el 'bibloquismo' y un voto por correo disparado\n", '\n', '\nÚLTIMA HORAEl juez envía a prisión provisional al asesino de Concha en Tirso de Molina y a su cómplice\n', '\nÚLTIMA HORAEl juez envía a prisión provisional al asesino de Concha en Ti

In [45]:
texts = ["".join([x.get_text() for x in div.find_all(search_p_tags_noattrs) if x != ""]) for div in divs_in_body]
texts

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '\n\n                                        Bomberos\n                                    \n\n\n                                        Rescate\n                                    \n\n\n                                        Aragón\n                                    \n\n\n                                        Lluvias torrenciales\n                                    \n\n\n                                        Zaragoza\n                                    \nAriesTauroGéminisCáncerLeoVirgoLibraEscorpioSagitarioCapricornioAcuarioPiscis',
 '\n\n                                        Bomberos\n                                    \n\n\n                                        Rescate\n                                    \n\n\n                                        Aragón\n                                    \n\n\n                                        Lluvias to

## Extract pdf and intellectual property

In [16]:
!pip install PyMuPDF

     ---------------------------------------- 11.8/11.8 MB 1.4 MB/s eta 0:00:00


In [17]:
pdf_file_name = './data/BOE-A-1996-8930-consolidado_ley propiedad intelectual.pdf'

In [49]:
ord("\U0001F600")

128512

In [52]:
chr(ord("\U0001F600"))

'😀'

In [48]:
print("\U0001F600")

😀


In [69]:
import fitz
import re

# Abrir el archivo PDF
doc = fitz.open(pdf_file_name)

# Iterar sobre todas las páginas y extraer el texto
text_cleaned = []
text_articles_cleaned = []
n_chars = 0
n_articles_chars = 0
for page in doc:
    text = page.get_text()
    # clean redundancy
    text = text.replace("BOLETÍN OFICIAL DEL ESTADO", "")
    text = text.replace("LEGISLACIÓN CONSOLIDADA", "")
    text = re.sub("[Pp]{1}[aá]gina [0-9]+", "", text)
    if "articulo" in text or "artículo" in text: "Articulo" in text or "Artículo" in text:
        text_articles_cleaned.append(text)
        n_articles_chars += len(text)
    text_cleaned.append(text)
    n_chars += len(text)
print(n_chars, n_articles_chars)

In [68]:
print(doc.get_page_text(50))

considerar que se trata de una nueva inversión sustancial, evaluada desde un punto de vista 
cuantitativo o cualitativo, permitirá atribuir a la base resultante de dicha inversión un plazo de 
protección propio.
Artículo 137.  Salvaguardia de aplicación de otras disposiciones.
Lo dispuesto en el presente Título se entenderá sin perjuicio de cualesquiera otras 
disposiciones legales que afecten a la estructura o al contenido de una base de datos tales 
como las relativas al derecho de autor u otros derechos de propiedad intelectual, al derecho 
de propiedad industrial, derecho de la competencia, derecho contractual, secretos, 
protección de los datos de carácter personal, protección de los tesoros nacionales o sobre el 
acceso a los documentos públicos.
LIBRO TERCERO
De la protección de los derechos reconocidos en esta Ley
TÍTULO I
Acciones y procedimientos
Artículo 138.  Acciones y medidas cautelares urgentes.
El titular de los derechos reconocidos en esta ley, sin perjuicio de otras a

## Extraccion de BOE texto refundido de propiedad intelectual

Xpath code to access paragraphs and article names:

    //*[@class="parrafo" or @class="articulo"]

In [72]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
url_boe = "https://www.boe.es/buscar/act.php?id=BOE-A-1996-8930&p=20220330&tn=1"
html = BeautifulSoup(requests.get(url_boe, headers=headers).content, "html.parser")

In [119]:
tags_with_text = html.find_all(attrs={"class": (re.compile(r"parrafo.*"), 
                                                re.compile(r"articulo.*"))})
separator = "\n"
texts_article = []
texts_paragraph = []
unexpected = []
texts = []
for x in tags_with_text:
    if any("articulo" in value for value in x.attrs["class"]):
        texts_article.append(x.get_text(separator))
    elif any("parrafo" in value for value in x.attrs["class"]):
        texts_paragraph.append(x.get_text(separator))
    else:
        print(x)
        unexpected.append(x.get_text(separator))
    texts.append(x.get_text(separator))
len(texts_article), len(texts_paragraph), len(texts),texts_article[:5], texts_paragraph[:5], unexpected

(248,
 1344,
 1592,
 ['Artículo único. Objeto de la norma.',
  'Disposición derogatoria única. Derogación normativa.',
  'Disposición final única. Entrada en vigor.',
  'Artículo 1. Hecho generador.',
  'Artículo 2. Contenido.'],
 ['La disposición final segunda de la Ley 27/1995, de 11 de octubre, de incorporación al Derecho español de la Directiva 93/98/CEE, del Consejo, de 29 de octubre, relativa a la armonización del plazo de protección del derecho de autor y de determinados derechos afines, autorizó al Gobierno para que, antes del 30 de junio de 1996, aprobara un texto que refundiese las disposiciones legales vigentes en materia de propiedad intelectual, regularizando, aclarando y armonizando los textos que hubieran de ser refundidos. El alcance temporal de esta habilitación legislativa es el relativo a las disposiciones legales que se encontrarán vigentes a 30 de junio de 1996.',
  'En consecuencia, se ha elaborado un texto refundido que se incorpora como anexo a este Real Decreto

In [121]:
texts_str = "\n".join(texts)
print(texts_str[:10000])

La disposición final segunda de la Ley 27/1995, de 11 de octubre, de incorporación al Derecho español de la Directiva 93/98/CEE, del Consejo, de 29 de octubre, relativa a la armonización del plazo de protección del derecho de autor y de determinados derechos afines, autorizó al Gobierno para que, antes del 30 de junio de 1996, aprobara un texto que refundiese las disposiciones legales vigentes en materia de propiedad intelectual, regularizando, aclarando y armonizando los textos que hubieran de ser refundidos. El alcance temporal de esta habilitación legislativa es el relativo a las disposiciones legales que se encontrarán vigentes a 30 de junio de 1996.
En consecuencia, se ha elaborado un texto refundido que se incorpora como anexo a este Real Decreto Legislativo, y que tiene por objeto dar cumplimiento al mandato legal.
En su virtud, a propuesta de la Ministra de Cultura, de acuerdo con el Consejo de Estado y previa deliberación del Consejo de Ministros en su reunión del día 12 de ab

In [127]:
with open("./data/BOE-A-1996-8930-consolidado_ley propiedad intelectual_plain text.txt", "w") as file:
    file.write("\n".join(texts))

In [4]:
if '':
    print(True)

## Pruebas urls

In [10]:
def read_media_urls_file(file_path):
    with open(file_path, "r") as file:
        return json.load(file)

In [151]:
def find_news(media_url, date, time):
    y, m = date.split("-")[:2]
    # With headers
    #response = requests.get(media_url, headers=HEADERS)
    try:
        response = requests.get(media_url, headers=HEADERS, timeout=10)  # Set an appropriate timeout value (in seconds)
    except requests.exceptions.Timeout:
        # Handle the timeout exception
        print("The request timed out.")
        return []
    except requests.exceptions.RequestException as e:
        # Handle other request exceptions
        print(f"An error occurred: {str(e)}")
        return []

    # HTTP status code
    #print(f"Status code {response.status_code} for {media_url}")

    parsed_hmtl = BeautifulSoup(response.content, "html.parser")

    tags_with_url = parsed_hmtl.find_all("a", href=re.compile("https?:.*"))
    
    tags_with_url = pd.Series(tags_with_url).unique().tolist()

    valid_news_urls = []
    for tag_with_url in tags_with_url:
        
        hiperlink = tag_with_url.attrs["href"]
        # Clean query ~Filter out urls with query symbols~
        if re.search("[?=&+%@#]{1}", hiperlink):
            #continue
            search_spec_char = re.search("[?=&+%@#]{1}", hiperlink)
            query_start = search_spec_char.span()[0]
            hiperlink = hiperlink[:query_start]
        if not re.search("/(\D+-+\D+-?)+/?", hiperlink) and "html" not in hiperlink:
            if "20minutos" in hiperlink:
                print("1- 20minutos not available", hiperlink)
            continue
        if media_url in hiperlink or media_url.replace("https://www.", "") in hiperlink or media_url.replace("http://www.", "") in hiperlink: 
            valid_news_urls.append(hiperlink)
        else:
            print("2- 20minutos not available", hiperlink)
        #else:
        #    print("\t\t Skipped from 1-find_news(): " + hiperlink)
                    
    return list(set(valid_news_urls))

In [83]:
pattern = "/(\D+-+\D+-?)+/?"

In [84]:
re.search(pattern, "-Asunción-")

In [85]:
re.search(pattern, "/dsds-Asunción-dwdds-jknvd-snd8/")

<re.Match object; span=(0, 30), match='/dsds-Asunción-dwdds-jknvd-snd'>

In [86]:
url = "https://www.elconfidencial.com/tecnologia/2023-07-04/cifrado-reglamento-europeo-abuso-menores-whatsapp-mensajeria_3684963/"
re.search(pattern, url)

<re.Match object; span=(52, 114), match='/cifrado-reglamento-europeo-abuso-menores-whatsap>

In [124]:
url = "https://okdiario.com/elecciones/sanchez-estalla-equipo-pifia-del-debate-rumbo-campana-estais-hundiendo-11241592"
re.search(pattern, url)
url[31:103]

'/sanchez-estalla-equipo-pifia-del-debate-rumbo-campana-estais-hundiendo-'

In [114]:
url = "https://www.20minutos.es/noticia/5146659/0/bertin-osborne-padre-los-69-anos-su-novia-gabriela-esta-embarazada-tres-meses/?homeAutoplay"
re.search(pattern, url)

<re.Match object; span=(42, 68), match='/bertin-osborne-padre-los-'>

In [121]:
url = "https://www.20minutos.es/noticia/5146585/0/tragedia-huelva-muere-ahogado-un-joven-23-anos-banarse-rio-tinto/"
re.search(pattern, url)

<re.Match object; span=(42, 68), match='/bertin-osborne-padre-los-'>

In [130]:
url = "https://www.20minutos.es/noticia/5146735/0/ayuso-ha-perdido-un-bebe-que-esperaba-con-ocho-semanas-gestacion-fue-intervenida-quirurgicamente-pasado-martes/"
re.search(pattern, url)

<re.Match object; span=(42, 154), match='/ayuso-ha-perdido-un-bebe-que-esperaba-con-ocho-s>

Other re

In [115]:
url

'https://www.20minutos.es/noticia/5146659/0/bertin-osborne-padre-los-69-anos-su-novia-gabriela-esta-embarazada-tres-meses/?homeAutoplay'

In [119]:
print(re.search("[?=&+%@#]{1}", url))
#continue
search_spec_char = re.search("[?=&+%@#]{1}", url)
query_start = search_spec_char.span()[0]
url_new = url[:query_start]
url_new

<re.Match object; span=(121, 122), match='?'>


'https://www.20minutos.es/noticia/5146659/0/bertin-osborne-padre-los-69-anos-su-novia-gabriela-esta-embarazada-tres-meses/'

In [149]:
url

'https://www.20minutos.es/noticia/5146735/0/ayuso-ha-perdido-un-bebe-que-esperaba-con-ocho-semanas-gestacion-fue-intervenida-quirurgicamente-pasado-martes/'

In [150]:
url = "https://www.20minutos.es/mujer/rebeca-marin-hablaran-nosotras-cuando-hayamos-muerto-5146486/"
if not re.search("/(\D+-+\D+-?)+/?", url) and "html" not in url:
    print("Not valid")

In [118]:
file_name = "spain_media name_to_url.json"
file_path = os.path.join(".", "data", file_name)
if os.path.exists(file_path):
    print("Reading file of urls of regions...")
    name_to_media_urls = read_media_urls_file(file_path)

Reading file of urls of regions...


In [12]:
name_to_media_urls

{'El País': 'elpais.com',
 'ABC ': 'abc.es',
 'El Español': 'elespanol.com',
 'El Mundo': 'elmundo.es',
 'La Vanguardia': 'lavanguardia.com',
 'El Periódico de Catalunya': 'elperiodico.com',
 'La Razón': 'larazon.es',
 '20 Minutos': '20minutos.es',
 'ABC de Sevilla': 'sevilla.abc.es/',
 'Europa Press': 'europapress.es',
 'elDiario.es': 'eldiario.es',
 'La Voz de Galicia ': 'lavozdegalicia.es',
 'El Correo Español': 'elcorreo.com',
 'Libertad Digital': 'libertaddigital.com',
 'SUR': 'diariosur.es',
 'Diario de Sevilla': 'diariodesevilla.es',
 'Crónica global': 'cronicaglobal.elespanol.com',
 'El Diario Vasco ': 'diariovasco.com',
 'La Verdad': 'laverdad.es',
 'Levante': 'levante-emv.com',
 'El Confidencial': 'elconfidencial.com',
 'Las Provincias': 'lasprovincias.es',
 'Heraldo de Aragón ': 'heraldo.es',
 'OK Diario': 'okdiario.com',
 'Ideal ': 'ideal.es',
 'Huffington Post España': 'huffingtonpost.es',
 'Faro de Vigo': 'farodevigo.es',
 'Canarias 7': 'canarias7.es',
 'El Periódico de E

In [162]:
media_urls_100_elements

['El País',
 'ABC ',
 'El Español',
 'El Mundo',
 'La Vanguardia',
 'El Periódico de Catalunya',
 'La Razón',
 '20 Minutos',
 'ABC de Sevilla',
 'Europa Press',
 'elDiario.es',
 'La Voz de Galicia ',
 'El Correo Español',
 'Libertad Digital',
 'SUR',
 'Diario de Sevilla',
 'Crónica global',
 'El Diario Vasco ',
 'La Verdad',
 'Levante',
 'El Confidencial',
 'Las Provincias',
 'Heraldo de Aragón ',
 'OK Diario',
 'Ideal ',
 'Huffington Post España',
 'Faro de Vigo',
 'Canarias 7',
 'El Periódico de España ',
 'La Información ',
 'Periodista Digital',
 'Publico',
 'DEIA ',
 'El Comercio',
 'Diario de Mallorca',
 'Hoy de Badajoz',
 'El Diario Montañes',
 'Información',
 'La Rioja ',
 'República',
 'Diario de Cádiz ',
 'El Plural',
 'Diario de Noticias',
 'La Nueva España',
 'Última Hora',
 'El Confidencial Digital',
 'El Día ',
 'La Opinión de Murcia ',
 'El Periódico de Aragón ',
 'El Periódico de Extremadura',
 'El Norte de Castilla',
 'Diario de Córdoba ',
 'La voz',
 'El Correo Galleg

In [163]:
name_to_media_urls

{'El País': 'elpais.com',
 'ABC ': 'abc.es',
 'El Español': 'elespanol.com',
 'El Mundo': 'elmundo.es',
 'La Vanguardia': 'lavanguardia.com',
 'El Periódico de Catalunya': 'elperiodico.com',
 'La Razón': 'larazon.es',
 '20 Minutos': '20minutos.es',
 'ABC de Sevilla': 'sevilla.abc.es/',
 'Europa Press': 'europapress.es',
 'elDiario.es': 'eldiario.es',
 'La Voz de Galicia ': 'lavozdegalicia.es',
 'El Correo Español': 'elcorreo.com',
 'Libertad Digital': 'libertaddigital.com',
 'SUR': 'diariosur.es',
 'Diario de Sevilla': 'diariodesevilla.es',
 'Crónica global': 'cronicaglobal.elespanol.com',
 'El Diario Vasco ': 'diariovasco.com',
 'La Verdad': 'laverdad.es',
 'Levante': 'levante-emv.com',
 'El Confidencial': 'elconfidencial.com',
 'Las Provincias': 'lasprovincias.es',
 'Heraldo de Aragón ': 'heraldo.es',
 'OK Diario': 'okdiario.com',
 'Ideal ': 'ideal.es',
 'Huffington Post España': 'huffingtonpost.es',
 'Faro de Vigo': 'farodevigo.es',
 'Canarias 7': 'canarias7.es',
 'El Periódico de E

In [164]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
current_date, current_time = str(datetime.today()).split(" ")
data = {}
data_invalid = {}
max_n_elements = 100
for i, (name, media_url) in enumerate(name_to_media_urls.items()):
    complete_media_url = "https://www." + media_url
    news = find_news(complete_media_url, current_date, current_time)
    if len(news) == 0:
        data_invalid[name] = media_url
    data[name] = news
    if i >= max_n_elements:
        break

2- 20minutos not available https://cadenaser.com/nacional/2023/07/12/lo-repiten-por-activa-y-por-pasiva-el-discurso-de-puertas-para-dentro-del-real-madrid-en-el-caso-mbappe-cadena-ser/
2- 20minutos not available https://cadenaser.com/nacional/2023/07/12/lo-repiten-por-activa-y-por-pasiva-el-discurso-de-puertas-para-dentro-del-real-madrid-en-el-caso-mbappe-cadena-ser/
2- 20minutos not available https://www.huffingtonpost.es/virales/australiana-llega-europa-calienta-dice-esto-gente-espanoles.html
2- 20minutos not available https://www.huffingtonpost.es/virales/australiana-llega-europa-calienta-dice-esto-gente-espanoles.html
2- 20minutos not available https://as.com/futbol/primera/el-deja-vu-de-reinier-n/
2- 20minutos not available https://as.com/futbol/primera/el-deja-vu-de-reinier-n/
2- 20minutos not available https://retinatendencias.com/civilizacion-perdida/la-alquimia-de-salvame-de-la-basura-al-oro-y-el-fin-del-gafapastismo/
2- 20minutos not available https://retinatendencias.com/civ

2- 20minutos not available https://www.santander.com/es/sala-de-comunicacion/los-temas-que-nos-mueven
2- 20minutos not available https://www.santander.com/es/sala-de-comunicacion/los-temas-que-nos-mueven
1- 20minutos not available https://www.20minutos.es/
1- 20minutos not available https://www.20minutos.es/set_edition/es/
1- 20minutos not available https://www.20minutos.es/set_edition/cat/
1- 20minutos not available https://www.20minutos.es/set_edition/mad/
1- 20minutos not available https://www.20minutos.es/nacional/
1- 20minutos not available https://www.20minutos.es/internacional/
1- 20minutos not available https://www.20minutos.es/deportes/
1- 20minutos not available https://www.20minutos.es/minuteca/opinion/
1- 20minutos not available https://www.20minutos.es/television/
1- 20minutos not available https://www.20minutos.es/gente/
1- 20minutos not available https://www.20minutos.es/tecnologia/
1- 20minutos not available https://www.20minutos.es/edicion_impresa/
1- 20minutos not ava

2- 20minutos not available https://www.infosalus.com/salud-investigacion/noticia-pueden-bacterias-boca-causar-alzheimer-20230712083549.html
2- 20minutos not available https://www.infosalus.com/salud-investigacion/noticia-pueden-bacterias-boca-causar-alzheimer-20230712083549.html
2- 20minutos not available https://www.notichile.cl/chile/noticia-chile-pesar-criticas-presidente-boric-inicia-gira-mas-semana-europa-20230712124134.html
2- 20minutos not available https://www.notichile.cl/chile/noticia-chile-pesar-criticas-presidente-boric-inicia-gira-mas-semana-europa-20230712124134.html
2- 20minutos not available https://www.notimerica.com/politica/noticia-cuba-eeuu-cuba-tilda-escalada-provocadora-entrada-submarino-nuclear-eeuu-bahia-guantanamo-20230712091635.html
2- 20minutos not available https://www.infosalus.com/farmacia/noticia-metilfenidato-modafinilo-dextroanfetamina-mejoran-rendimiento-20230712081952.html
2- 20minutos not available https://www.infosalus.com/salud-investigacion/notici

2- 20minutos not available https://www.autocasion.com/coches-segunda-mano/malaga
2- 20minutos not available https://www.relevo.com/tenis/wimbledon/abucheos-disgustos-guerra-comunicados-tenis-20230711160835-nt.html
2- 20minutos not available https://www.relevo.com/tenis/wimbledon/abucheos-disgustos-guerra-comunicados-tenis-20230711160835-nt.html
2- 20minutos not available https://www.relevo.com/futbol/primera-rfef/antonio-torre-renuncia-cache-darle-20230711175357-nt.html
2- 20minutos not available https://www.relevo.com/futbol/primera-rfef/antonio-torre-renuncia-cache-darle-20230711175357-nt.html
2- 20minutos not available https://www.relevo.com/ultima-hora/
2- 20minutos not available https://www.relevo.com/ciclismo/tour-francia/
2- 20minutos not available https://www.relevo.com/futbol/futbol-femenino/
2- 20minutos not available https://www.relevo.com/futbol/mercado-fichajes/
2- 20minutos not available https://laboratoriodeanalisisdealimentosyaguas.atencionweb.es/laboratorio-de-analisis

2- 20minutos not available http://www.autocasion.com/coches-segunda-mano/murcia
2- 20minutos not available https://www.relevo.com/tenis/wimbledon/abucheos-disgustos-guerra-comunicados-tenis-20230711160835-nt.html
2- 20minutos not available https://www.relevo.com/tenis/wimbledon/abucheos-disgustos-guerra-comunicados-tenis-20230711160835-nt.html
2- 20minutos not available https://www.relevo.com/futbol/primera-rfef/antonio-torre-renuncia-cache-darle-20230711175357-nt.html
2- 20minutos not available https://www.relevo.com/futbol/primera-rfef/antonio-torre-renuncia-cache-darle-20230711175357-nt.html
2- 20minutos not available https://www.relevo.com/ultima-hora/
2- 20minutos not available https://www.relevo.com/ciclismo/tour-francia/
2- 20minutos not available https://www.relevo.com/futbol/futbol-femenino/
2- 20minutos not available https://www.relevo.com/futbol/mercado-fichajes/
2- 20minutos not available https://www.mujerhoy.com/celebrities/anna-kournikova-cunada-isabel-preysler-boda-enriq

...

2- 20minutos not available https://sevilla.abc.es/andalucia/guerra-agua-sacude-campana-electoral-andalucia-20230710220520-nts.html
2- 20minutos not available https://www.abc.es/espana/sanchez-frena-ascenso-derecha-sigue-encima-180-20230709205441-nt.html
2- 20minutos not available https://sevilla.abc.es/orgullodenervion/noticias-sevilla-fc/sevi-junta-general-accionistas-2022-sevilla-directo-202212291558_directo.html
2- 20minutos not available https://sevilla.abc.es/andalucia/loteria-navidad-2022-andalucia-sigue-directo-sorteo-gordo-20221222070856-dis.html
2- 20minutos not available https://www.abc.es/loteria-de-navidad/comprobar-loteria-navidad/
2- 20minutos not available https://sevilla.abc.es/elecciones/andalucia/sevi-elecciones-andalucia-2022-resultados-sondeo-ganador-participacion-escrutinio-202206190803_directo.html
2- 20minutos not available https://sevilla.abc.es/elecciones/andalucia/sevi-elecciones-andalucia-2022-directo-pp-coloca-escanos-mayoria-absoluta-202206131012_directo.ht

2- 20minutos not available https://www.iberempleos.es/ofertas-empleo/a-coruna
2- 20minutos not available https://www.tucasa.com/compra-venta/viviendas/a-coruna/
2- 20minutos not available https://www.tucasa.com/alquiler/viviendas/a-coruna/
2- 20minutos not available https://ocasion.neomotor.com/coches-segunda-mano-a-coruna
2- 20minutos not available https://www.compramejor.es/mejores-ofertas-tecnologia-prime-day-2023/
2- 20minutos not available https://www.compramejor.es/mejores-ofertas-tecnologia-prime-day-2023/
2- 20minutos not available https://www.compramejor.es/productos-sorprendentes-amazon-prime-day/
2- 20minutos not available https://www.compramejor.es/productos-sorprendentes-amazon-prime-day/
2- 20minutos not available https://www.epe.es/es/activos/20230712/prefiere-paradores-loterias-o-hipodromo-89741771
2- 20minutos not available https://www.epe.es/es/activos/20230712/prefiere-paradores-loterias-o-hipodromo-89741771
2- 20minutos not available https://www.epe.es/es/activos/20

2- 20minutos not available https://www.tucasa.com/compra-venta/viviendas/las-palmas/
2- 20minutos not available https://www.tucasa.com/alquiler/viviendas/las-palmas/
2- 20minutos not available https://www.iberempleos.es/ofertas-empleo/las-palmas
2- 20minutos not available https://ocasion.neomotor.com/coches-segunda-mano-las-palmas
2- 20minutos not available https://www.linkedin.com//company/la-provincia-diario-de-las-palmas
2- 20minutos not available https://www.compramejor.es/mejores-ofertas-tecnologia-prime-day-2023/
2- 20minutos not available https://www.compramejor.es/mejores-ofertas-tecnologia-prime-day-2023/
2- 20minutos not available https://www.compramejor.es/productos-sorprendentes-amazon-prime-day/
2- 20minutos not available https://www.compramejor.es/productos-sorprendentes-amazon-prime-day/
2- 20minutos not available https://www.epe.es/es/activos/20230712/prefiere-paradores-loterias-o-hipodromo-89741771
2- 20minutos not available https://www.epe.es/es/activos/20230712/prefi

2- 20minutos not available https://www.linkedin.com/company/madridiario-sl
2- 20minutos not available https://www.linkedin.com/company/madridiario-sl
2- 20minutos not available https://www.estrategiasdeinversion.com/analisis/bolsa-y-mercados/informes/por-que-sube-bitcoin-en-la-inversion-institucional-n-632769
2- 20minutos not available https://www.estrategiasdeinversion.com/actualidad/noticias/bolsa-eeuu/warren-buffett-no-le-teme-a-los-combustibles-fosiles-n-632639
An error occurred: HTTPSConnectionPool(host='www.diariodevalladolid.elmundo.es', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002B3160AA520>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
2- 20minutos not available https://www.aemet.es/es/eltiempo/prediccion/municipios/salamanca-id37274
2- 20minutos not available https://www.aemet.es/es/eltiempo/prediccion/municipios/salamanca-id37274
2- 20minutos not available https:

2- 20minutos not available https://es.linkedin.com/company/atlantico-diario
2- 20minutos not available https://www.unia.es/estudios-y-acceso/oferta-academica/cursos-de-verano
2- 20minutos not available https://www.unia.es/estudios-y-acceso/oferta-academica/masteres-oficiales
2- 20minutos not available https://club.heraldo.es/elige-tu-suscripcion/
1- 20minutos not available https://www.20minutos.es/
1- 20minutos not available https://www.20minutos.es/cinemania/
2- 20minutos not available https://www.cofciudadreal.com/paginas/Farmacias-Guardia.asp
2- 20minutos not available https://www.cofciudadreal.com/paginas/Farmacias-Guardia.asp
2- 20minutos not available https://www.dieselogasolina.com/gasolineras-en-ciudad-real.html
2- 20minutos not available https://www.dieselogasolina.com/gasolineras-en-ciudad-real.html
2- 20minutos not available https://www.loteriasyapuestas.es/es/loteria-nacional
2- 20minutos not available https://www.loteriasyapuestas.es/es/loteria-nacional
2- 20minutos not av

In [138]:
hiperlink = "https://www.20minutos.es/noticia/5146398/0/colas-correos-acaba-plazo-solicitar-voto-correo-oficinas-usuarios/"
re.search("[?=&+%@#]{1}", hiperlink)

In [2]:
data

NameError: name 'data' is not defined

In [165]:
len(data)

101

In [166]:
freq_dict_news = {k: len(v) for k, v in data.items()}
freq_news = [len(v) for k, v in data.items()]
sum(freq_news)

8842

In [167]:
sum([x == 0 for x in freq_news])

9

In [156]:
freq_dict_news

{'El País': 246,
 'ABC ': 61,
 'El Español': 75,
 'El Mundo': 217,
 'La Vanguardia': 37,
 'El Periódico de Catalunya': 272,
 'La Razón': 228,
 '20 Minutos': 142,
 'ABC de Sevilla': 86,
 'Europa Press': 116,
 'elDiario.es': 96,
 'La Voz de Galicia ': 71,
 'El Correo Español': 167,
 'Libertad Digital': 120,
 'SUR': 137,
 'Diario de Sevilla': 27,
 'Crónica global': 0,
 'El Diario Vasco ': 141,
 'La Verdad': 101,
 'Levante': 145,
 'El Confidencial': 161,
 'Las Provincias': 123,
 'Heraldo de Aragón ': 24,
 'OK Diario': 246,
 'Ideal ': 105,
 'Huffington Post España': 28,
 'Faro de Vigo': 95,
 'Canarias 7': 83,
 'El Periódico de España ': 186,
 'La Información ': 57,
 'Periodista Digital': 171,
 'Publico': 81,
 'DEIA ': 58,
 'El Comercio': 172,
 'Diario de Mallorca': 84,
 'Hoy de Badajoz': 107,
 'El Diario Montañes': 146,
 'Información': 112,
 'La Rioja ': 104,
 'República': 90,
 'Diario de Cádiz ': 20,
 'El Plural': 103,
 'Diario de Noticias': 84,
 'La Nueva España': 84,
 'Última Hora': 16,


In [168]:
data_invalid

{'Crónica global': 'cronicaglobal.elespanol.com',
 'Heraldo diario de soria': 'heraldodiariodesoria.elmundo.es',
 'El correo de burgos': 'elcorreodeburgos.elmundo.es',
 'Diari de Tarragona': 'diaridetarragona.com',
 'La crónica de badajoz': 'lacronicadebadajoz.elperiodicoextremadura.com',
 'Diario de Valladolid': 'diariodevalladolid.elmundo.es',
 'Diario de Burgos': 'diariodeburgos.es',
 'Diario siglo xxi': 'diariosigloxxi.com',
 'El correo de andalucía': 'elcorreoweb.es'}

## General requests

In [136]:
url = "https://www.vozpopuli.com/espana/pp-aprieta-abascal-feijoo.html"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=HEADERS)

In [137]:
response.text

'<!doctype html>\n<html lang="es">\n<head>\n<meta charset="UTF-8">\n<meta name="viewport" content="width=device-width, user-scalable=no, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0">\n<meta http-equiv="X-UA-Compatible" content="ie=edge">\n<title>El PP aprieta a Abascal tras la reunión con Feijóo para que el interlocutor sea Espinosa de los Monteros y no Buxadé</title>\n<link rel="shortcut icon" href="https://www.vozpopuli.com/wp-content/themes/vozpopulicom/assets/img/vp_400x400-150x150.png" type="image/x-icon" sizes="32x32" />\n<link rel="shortcut icon" href="https://www.vozpopuli.com/wp-content/themes/vozpopulicom/assets/img/vp_400x400-235x235.png" type="image/x-icon" sizes="192x192" />\n<link rel="apple-touch-icon" href="https://www.vozpopuli.com/wp-content/themes/vozpopulicom/assets/img/vp_400x400-235x235.png">\n<meta name="msapplication-TileImage" content="https://www.vozpopuli.com/wp-content/themes/vozpopulicom/assets/img/vp_400x400-300x300.png">\n<meta name="lang" con

In [138]:
parsed_hmtl = BeautifulSoup(response.content, "html.parser")
tags_with_text = parsed_hmtl.find("body").find_all(lambda tag: (tag.name == "p" and not tag.attrs) or ("h" in tag.name and not tag.attrs))
text_clean_from_tags = "".join([re.sub("\n+", "\n", tag.get_text()) for tag in tags_with_text])

In [177]:
print(text_clean_from_tags)

DESTACADO
España 

Jesús Ortega 
Publicado: 01/08/2023 04:45Actualizado: 01/08/2023 04:50El PP ve más cerca un apoyo de Vox a la investidura tras la reunión de Feijóo con Abascal, pero en Génova no se fían. Las negociaciones para conformar gobiernos autonómicos, especialmente en el caso de Extremadura, han confirmado a los populares que el sector "más radical" del partido, liderado por Jorge Buxadé, puede hacer peligrar cualquier acuerdo. Por ese motivo, los populares aprietan a Abascal para que el interlocutor en los próximos contactos sea Iván Espinosa de los Monteros, cuyo papel ya fue clave en pactos pasados.Esa hoja de ruta explica algunas declaraciones de altos dirigentes del PP en los últimos días, donde no se ha rebajado el tono hacia Vox pese a la reunión entre Feijóo y Abascal: "Si estamos en la situación que estamos es por su caída fruto de una malísima campaña pilotada por Buxadé. Ahora, las posibilidades que tengamos pasan por un cambio de estrategia de Vox, con nuevos act

In [199]:
meta_data = parsed_hmtl.select("html head meta[property]")
meta_data = {"-".join(x.attrs["property"].split(":")[1:]) + "_" + str(i): x.attrs["content"] for i, x in enumerate(meta_data)}

raw_keys_data = {k: v for k, v in meta_data.items() if k.split("_")[0] in keys}
keys_data = {}
keys_data["title"] = raw_keys_data

{'title_2': 'El PP aprieta a Abascal tras la reunión con Feijóo para que el interlocutor sea Espinosa de los Monteros y no Buxadé',
 'tag_6': 'Alberto Núñez Feijóo',
 'tag_7': 'Cuca Gamarra',
 'tag_8': 'Iván Espinosa de los Monteros',
 'tag_9': 'PP (Partido Popular)',
 'tag_10': 'Santiago Abascal',
 'tag_11': 'Vox',
 'image_14': 'https://media.vozpopuli.com/2023/07/feijooabascal.jpg'}

In [198]:
meta_data

{'locale_0': 'es_ES',
 'type_1': 'article',
 'title_2': 'El PP aprieta a Abascal tras la reunión con Feijóo para que el interlocutor sea Espinosa de los Monteros y no Buxadé',
 'description_3': 'Génova espera que la negociación con Vox para el apoyo a la investidura avance con el sector más moderado del partido, con el que se llegó a acuerdos en el pasado, y no con "quien ha querido romper todos los puentes"',
 'url_4': 'https://www.vozpopuli.com/espana/pp-aprieta-abascal-feijoo.html',
 'site_name_5': 'Vozpópuli',
 'tag_6': 'Alberto Núñez Feijóo',
 'tag_7': 'Cuca Gamarra',
 'tag_8': 'Iván Espinosa de los Monteros',
 'tag_9': 'PP (Partido Popular)',
 'tag_10': 'Santiago Abascal',
 'tag_11': 'Vox',
 'section_12': 'España',
 'updated_time_13': '2023-08-01T04:50:05+02:00',
 'image_14': 'https://media.vozpopuli.com/2023/07/feijooabascal.jpg',
 'image-secure_url_15': 'https://media.vozpopuli.com/2023/07/feijooabascal.jpg',
 'image-width_16': '1200',
 'image-height_17': '675',
 'image-alt_18'

In [225]:
re.search("application[/]{1}ld[+]{1}json", "script type='application/ld+json'")

<re.Match object; span=(13, 32), match='application/ld+json'>

In [216]:
meta_data = parsed_hmtl.select("html head meta[property]")
target_keys = ("title", "published_time", "modified_time", "tag", "image")
data = {}
tags = []
for tag in meta_data:
    _property = tag.attrs["property"].split(":")[-1]
    content = tag.attrs["content"]
    if "tag" in _property:
        tags.append(content)
    elif _property in target_keys:
        data[_property] = content
data["tags"] = tags

In [217]:
meta_data

[<meta content="es_ES" property="og:locale"/>,
 <meta content="article" property="og:type"/>,
 <meta content="El PP aprieta a Abascal tras la reunión con Feijóo para que el interlocutor sea Espinosa de los Monteros y no Buxadé" property="og:title"/>,
 <meta content='Génova espera que la negociación con Vox para el apoyo a la investidura avance con el sector más moderado del partido, con el que se llegó a acuerdos en el pasado, y no con "quien ha querido romper todos los puentes"' property="og:description"/>,
 <meta content="https://www.vozpopuli.com/espana/pp-aprieta-abascal-feijoo.html" property="og:url"/>,
 <meta content="Vozpópuli" property="og:site_name"/>,
 <meta content="Alberto Núñez Feijóo" property="article:tag"/>,
 <meta content="Cuca Gamarra" property="article:tag"/>,
 <meta content="Iván Espinosa de los Monteros" property="article:tag"/>,
 <meta content="PP (Partido Popular)" property="article:tag"/>,
 <meta content="Santiago Abascal" property="article:tag"/>,
 <meta conten

In [218]:
tag

<meta content="true" property="ya:ovs:allow_embed"/>

In [219]:
data

{'title': 'El PP aprieta a Abascal tras la reunión con Feijóo para que el interlocutor sea Espinosa de los Monteros y no Buxadé',
 'image': 'https://media.vozpopuli.com/2023/07/feijooabascal.jpg',
 'published_time': '2023-08-01T04:45:00+02:00',
 'modified_time': '2023-08-01T04:50:05+02:00',
 'tags': ['Alberto Núñez Feijóo',
  'Cuca Gamarra',
  'Iván Espinosa de los Monteros',
  'PP (Partido Popular)',
  'Santiago Abascal',
  'Vox']}

In [ ]:
meta = parsed_hmtl.find_all(By.XPATH, "html/head/meta[@property]")
meta_data = {x.get_attribute("property").split(":")[-1]: x.get_attribute("content") for x in meta}


In [160]:
parsed_hmtl = BeautifulSoup(response.content, "lxml")
tags_with_text = parsed_hmtl.select("p")
text_clean_from_tags = "".join([re.sub("\n+", "\n", tag.get_text()) for tag in tags_with_text])

In [161]:
tags_with_text

[<p class="title-destacado">DESTACADO</p>,
 <p class="post-title-pre-header">
 <a href="https://www.vozpopuli.com/espana" rel="category tag">
 España </a>
 </p>,
 <p class="author-names">
 <span class="author-names-item">
 <a class="link-author" href="https://www.vozpopuli.com/redaccion/jesus-ortega" rel="author">
 Jesús Ortega </a>
 </span>
 </p>,
 <p class="date-group-published">Publicado: 01/08/2023 04:45</p>,
 <p class="date-group-updated">Actualizado: 01/08/2023 04:50</p>,
 <p>El <a data-id="www.pp.es" data-type="URL" href="http://www.pp.es" rel="noreferrer noopener" target="_blank">PP</a> ve más cerca un apoyo de Vox a la investidura tras la <a data-id="https://www.vozpopuli.com/espana/politica/elecciones-generales/feijoo-abascal-reunion-semana-pasada.html" data-type="URL" href="https://www.vozpopuli.com/espana/politica/elecciones-generales/feijoo-abascal-reunion-semana-pasada.html" rel="noreferrer noopener" target="_blank">reunión de Feijóo con Abascal</a>, pero en Génova no se 

In [163]:
print(text_clean_from_tags)

DESTACADO
España 

Jesús Ortega 
Publicado: 01/08/2023 04:45Actualizado: 01/08/2023 04:50El PP ve más cerca un apoyo de Vox a la investidura tras la reunión de Feijóo con Abascal, pero en Génova no se fían. Las negociaciones para conformar gobiernos autonómicos, especialmente en el caso de Extremadura, han confirmado a los populares que el sector "más radical" del partido, liderado por Jorge Buxadé, puede hacer peligrar cualquier acuerdo. Por ese motivo, los populares aprietan a Abascal para que el interlocutor en los próximos contactos sea Iván Espinosa de los Monteros, cuyo papel ya fue clave en pactos pasados.Esa hoja de ruta explica algunas declaraciones de altos dirigentes del PP en los últimos días, donde no se ha rebajado el tono hacia Vox pese a la reunión entre Feijóo y Abascal: "Si estamos en la situación que estamos es por su caída fruto de una malísima campaña pilotada por Buxadé. Ahora, las posibilidades que tengamos pasan por un cambio de estrategia de Vox, con nuevos act

In [45]:
url = "https://okdiario.com/espana/psoe-asume-que-cesiones-junts-superaran-erc-busca-formulas-camuflarlas-11333749"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=HEADERS)

parsed_hmtl = BeautifulSoup(response.content, "html.parser")
tags_with_text = parsed_hmtl.find("body").find_all(lambda tag: (tag.name == "p" and not tag.attrs) or ("h" in tag.name and not tag.attrs))
text_clean_from_tags = "".join([re.sub("\n+", "\n", tag.get_text()) for tag in tags_with_text])

In [38]:
print(text_clean_from_tags)

En el PSOE admiten que "hay que cuidar el lenguaje" y buscar fórmulas imaginativas para dar rienda suelta a las cesiones que pide JuntsJunts y ERC ya negocian el precio conjunto que quieren obligar hacer pagar a Sánchez para investirleSánchez cederá el «referéndum consultivo» que figura en la Constitución pero Puigdemont lo exige yaLa misma fórmula que con el PNV y Bildu. Pero a la catalana, con ERC y con Junts. Para garantizar la investidura de Pedro Sánchez el PSOE asume que en el proceso de negociación para asegurarse el apoyo de los de Carles Puigdemont tendrá que asumir una serie de cesiones del PSOE a Junts, en función de las exigencias de éstos, que pueden no gustar en su socio hasta ahora, ERC. Ferraz y Moncloa descartan que una de ellas pueda ser la amnistía, como exige Puigdemont, pero como con el referéndum consultivo «hay que tirar de imaginación» para que se puedan hacer concesiones «dentro del margen de la Constitución». En el equipo del presidente advierten a los negocia

In [39]:
text_clean_from_tags.split("\n")[0]

'En el PSOE admiten que "hay que cuidar el lenguaje" y buscar fórmulas imaginativas para dar rienda suelta a las cesiones que pide JuntsJunts y ERC ya negocian el precio conjunto que quieren obligar hacer pagar a Sánchez para investirleSánchez cederá el «referéndum consultivo» que figura en la Constitución pero Puigdemont lo exige yaLa misma fórmula que con el PNV y Bildu. Pero a la catalana, con ERC y con Junts. Para garantizar la investidura de Pedro Sánchez el PSOE asume que en el proceso de negociación para asegurarse el apoyo de los de Carles Puigdemont tendrá que asumir una serie de cesiones del PSOE a Junts, en función de las exigencias de éstos, que pueden no gustar en su socio hasta ahora, ERC. Ferraz y Moncloa descartan que una de ellas pueda ser la amnistía, como exige Puigdemont, pero como con el referéndum consultivo «hay que tirar de imaginación» para que se puedan hacer concesiones «dentro del margen de la Constitución». En el equipo del presidente advierten a los negoci

In [242]:
parsed_hmtl.html.attrs["lang"]

'es'

In [244]:
x = {"a": 1, "b": 2, "c": 3}
x.update({"a": 222, "d": 117, "c": 4})
x

{'a': 222, 'b': 2, 'c': 4, 'd': 117}

### Selenium

In [229]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# Configure Chrome to run in headless mode
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration, which is necessary for headless mode on some platforms
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.1234.567 Safari/537.36"
)
caps = DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "none"  # Do not wait for full page load

driver = webdriver.Chrome(options=chrome_options)

url = "https://www.vozpopuli.com/espana/pp-aprieta-abascal-feijoo.html"
#url = "https://okdiario.com/espana/marlaska-oculto-hasta-despues-del-23j-informe-que-revela-aumento-delincuencia-11353052"
#url = "https://www.huffingtonpost.es/tiempo/la-aemet-advierte-desplome-inusual-temperaturas-dia-semanabr.html"
#url = "https://www.lavanguardia.com/politica/20230731/9142911/lucha-tactica-iniciativa-sanchez-feijoo-sendas-cartas.html"
#url = "https://edition.cnn.com/2023/08/02/americas/mexico-cloud-seeding-drought-rain-climate-intl/index.html"
driver.get(url)

In [230]:
paragraphs = driver.find_elements(By.XPATH, "html/body//div/p|h1|h2")
len(paragraphs)

51

In [232]:
paragraphs = driver.find_elements(By.XPATH, "html/body//p|h1|h2")
print(len(paragraphs))
#parag_texts = {(i, x.tag_name): x.text if x.text else "\n"  for i, x in enumerate(paragraphs)}
parag_texts = {i: x.text if x.text else "\n"  for i, x in enumerate(paragraphs)}
parag_texts

55


{0: 'DESTACADO',
 1: 'ESPAÑA',
 2: 'Jesús Ortega',
 3: 'Publicado: 01/08/2023 04:45',
 4: 'Actualizado: 01/08/2023 04:50',
 5: 'El PP ve más cerca un apoyo de Vox a la investidura tras la reunión de Feijóo con Abascal, pero en Génova no se fían. Las negociaciones para conformar gobiernos autonómicos, especialmente en el caso de Extremadura, han confirmado a los populares que el sector "más radical" del partido, liderado por Jorge Buxadé, puede hacer peligrar cualquier acuerdo. Por ese motivo, los populares aprietan a Abascal para que el interlocutor en los próximos contactos sea Iván Espinosa de los Monteros, cuyo papel ya fue clave en pactos pasados.',
 6: 'Esa hoja de ruta explica algunas declaraciones de altos dirigentes del PP en los últimos días, donde no se ha rebajado el tono hacia Vox pese a la reunión entre Feijóo y Abascal: "Si estamos en la situación que estamos es por su caída fruto de una malísima campaña pilotada por Buxadé. Ahora, las posibilidades que tengamos pasan por

In [228]:
driver.quit()

In [234]:
meta = driver.find_elements(By.XPATH, "html/head/meta[@property]")
meta_data = {x.get_attribute("property").split(":")[-1]: x.get_attribute("content") for x in meta}
keys = ("title", "published_time", "modified_time", "tag", "image")
data = {k: v for k, v in meta_data.items() if k in keys}

In [131]:
driver.find_elements(By.XPATH, "html/head/script[contains(@type, '+json')]")[0]

'application/ld+json'

In [238]:
str(data)[1:-1]

"'title': 'El PP aprieta a Abascal tras la reunión con Feijóo para que el interlocutor sea Espinosa de los Monteros y no Buxadé', 'tag': 'Vox', 'image': 'https://media.vozpopuli.com/2023/07/feijooabascal.jpg', 'published_time': '2023-08-01T04:45:00+02:00', 'modified_time': '2023-08-01T04:50:05+02:00'"

## Test urls news extraction

In [1]:
import time

In [2]:
time.perf_counter()

254.2073487

## Look for links from main page headers

In [22]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
url = "https://www.lavanguardia.com/"
url = "https://www.abc.es/"
url = "https://www.elpais.com"
url = "https://republica.com/"
url = "https://www.huffingtonpost.es/"
response = requests.get(url, headers=headers)

parsed_hmtl = BeautifulSoup(response.content, "html.parser")

In [23]:
from datetime import datetime, timedelta, timezone
iso_datetime = datetime.now().replace(tzinfo=timezone.utc)
iso_datetime.strftime("%Y-%m-%d %H:%M:%S")

'2023-09-09 11:49:42'

In [24]:
response.url

'https://www.huffingtonpost.es/'

In [25]:
response.url.split("/")[2], \
"/".join(response.url.split("/")[:3] + [""]), \
"/".join(url.split("/")[:3] + [""])

('www.huffingtonpost.es',
 'https://www.huffingtonpost.es/',
 'https://www.huffingtonpost.es/')

In [26]:
li_tags = parsed_hmtl.header.find_all("li")
links = [x.find("a").get("href", None) for x in li_tags if x.find("a")]
len(links), links[:10]

(0, [])

In [27]:
len(links), links[:10]

(0, [])

In [84]:
(href and href != response.url and response.url in href)

False

In [92]:
a_tags = parsed_hmtl.html.body.find_all("a")
links = []
for a_tag in a_tags:
    href = a_tag.attrs.get("href", "")
    if not href or href == response.url:
        continue
    if (response.url in href) or href.startswith("/"):
        link_no_domain = href.replace(response.url, "/")
        link_search = re.search("(/[a-zA-Z0-9]+-?([a-zA-Z0-9]+)?/?)/?$", link_no_domain)
        if link_search:
            if not href.startswith(response.url):
                href = response.url[:-1] + href
            if not href.endswith("/"):
                href = href + "/"
            links.append(href)
len(list(set(links))), list(set(links))

(32,
 ['https://www.huffingtonpost.es/life/hijos/',
  'https://www.huffingtonpost.es/pillalo/',
  'https://www.huffingtonpost.es/life/consumo/',
  'https://www.huffingtonpost.es/life/influencers-celebrities/',
  'https://www.huffingtonpost.es/life/cultura/',
  'https://www.huffingtonpost.es/life/viajes/',
  'https://www.huffingtonpost.es/author/abraham-garcia/',
  'https://www.huffingtonpost.es/deporte/',
  'https://www.huffingtonpost.es/deporte/kings-league/',
  'https://www.huffingtonpost.es/virales/',
  'https://www.huffingtonpost.es/global/',
  'https://www.huffingtonpost.es/opinion/',
  'https://www.huffingtonpost.es/planeta/',
  'https://www.huffingtonpost.es/author/andres-lomena/',
  'https://www.huffingtonpost.es/medios/',
  'https://www.huffingtonpost.es/elecciones/generales/',
  'https://www.huffingtonpost.es/life/territorio-paradores/',
  'https://www.huffingtonpost.es/life/salud/',
  'https://www.huffingtonpost.es/author/uxia-prieto/',
  'https://www.huffingtonpost.es/polit

In [75]:
re.search("(/[a-zA-Z0-9]+){1,2}/?", "/elecciones/generales/")

<re.Match object; span=(0, 22), match='/elecciones/generales/'>

In [36]:
links_ser = pd.Series(links)
links_ser.str.split("/")

0     [, global, terremoto-marruecos-hoy-directo-ult...
1     [, loterias, loteria-nacional-hoy-sabado-9-sep...
2     [, tiempo, la-aemet-advierte-viene-resaca-fran...
3      [, economia, seis-cosas-cubre-seguro-hogar.html]
4        [, global, la-armada-espanola-pies-rusia.html]
5     [, economia, idealista-necesita-vender-casas-2...
6     [, sociedad, un-pais-inesperado-emerge-fuerza-...
7     [, sociedad, este-sueldo-profesor-universidad-...
8     [, economia, estos-son-nuevos-jefazos-mercadon...
9     [, economia, subida-pensiones-2024-estas-son-m...
10    [, sociedad, 10-apellidos-valencianos-indican-...
11                                         [, politica]
12                                           [, global]
13                                          [, virales]
14                                             [, life]
15                                           [, videos]
16                                 [, ultimas-noticias]
17                            [, static, contact

In [82]:
links_serie = pd.Series(links).dropna()
nodes = links_serie.str.replace(url, "", regex=True)

nodes_split = nodes.str.split("/")
nodes_split_clean = nodes_split.apply(lambda x: [elem for elem in x if elem])

nodes_split_clean_filter = nodes_split_clean.str.len().eq(1)

valid_links = links_serie[nodes_split_clean_filter]
valid_links

0             https://www.huffingtonpost.es/politica/
1               https://www.huffingtonpost.es/global/
2              https://www.huffingtonpost.es/virales/
3                 https://www.huffingtonpost.es/life/
4               https://www.huffingtonpost.es/videos/
5     https://www.huffingtonpost.es/ultimas-noticias/
6             https://www.huffingtonpost.es/politica/
8             https://www.huffingtonpost.es/economia/
9               https://www.huffingtonpost.es/global/
10             https://www.huffingtonpost.es/opinion/
11               https://www.huffingtonpost.es/lgtbi/
12             https://www.huffingtonpost.es/planeta/
13          https://www.huffingtonpost.es/tecnologia/
14             https://www.huffingtonpost.es/deporte/
16              https://www.huffingtonpost.es/medios/
17                https://www.huffingtonpost.es/life/
27             https://www.huffingtonpost.es/virales/
28            https://www.huffingtonpost.es/sociedad/
29              https://www.

hrefs from node1

In [5]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
url = "https://www.lavanguardia.com/magazine"
url = "https://www.abc.es/economia"
response = requests.get(url, headers=headers)

parsed_hmtl = BeautifulSoup(response.content, "html.parser")
node1_a_tags = parsed_hmtl.find_all("a", href=re.compile("/+"))
node1_links = [x.attrs["href"] for x in node1_a_tags]

hrefs from node2 from node2

In [6]:
node1_links_serie = pd.Series(node1_links)
node1_links_serie

0      https://www.abc.es/sociedad/aemet-fecha-vuelta...
1      https://www.abc.es/internacional/asesinan-deca...
2      https://www.abc.es/gente/nueva-vida-errante-to...
3      https://www.abc.es/espana/madrid/fiestas-san-c...
4      https://www.abc.es/viajar/destinos/espana/carr...
                             ...                        
291                          https://www.autocasion.com/
292                               https://www.pisos.com/
293                             https://www.unoauto.com/
294                          https://www.infoempleo.com/
295                               https://www.welife.es/
Length: 296, dtype: object

In [7]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
url = "https://www.lavanguardia.com/magazine/viajes"
url = "https://www.abc.es/economia/inmobiliario"
response = requests.get(url, headers=headers)

parsed_hmtl = BeautifulSoup(response.content, "html.parser")
node2_a_tags = parsed_hmtl.find_all("a", href=re.compile("/+"))
node2_links = [x.attrs["href"] for x in node2_a_tags]

In [8]:
node2_links_serie = pd.Series(node2_links)
node2_links_serie

0      https://www.abc.es/sociedad/aemet-fecha-vuelta...
1      https://www.abc.es/internacional/asesinan-deca...
2      https://www.abc.es/gente/nueva-vida-errante-to...
3      https://www.abc.es/espana/madrid/fiestas-san-c...
4      https://www.abc.es/viajar/destinos/espana/carr...
                             ...                        
235                          https://www.autocasion.com/
236                               https://www.pisos.com/
237                             https://www.unoauto.com/
238                          https://www.infoempleo.com/
239                               https://www.welife.es/
Length: 240, dtype: object

In [9]:
node1_links_serie = pd.DataFrame(node1_links, 
                              columns=["url1"]
                              )
node2_links_serie = pd.DataFrame(node2_links, 
                              columns=["url2"]
                              )

are_new_news = node1_links_serie.merge(node2_links_serie, 
                                left_on="url1", 
                                right_on="url2", 
                                how="left").isnull().any(axis=1)

In [10]:
are_new_news.sum()

92

In [11]:
pd.concat([node1_links_serie, 
           node2_links_serie], axis=1)

,url1,url2
0,https://www.abc.es/sociedad/aemet-fecha-vuelta...,https://www.abc.es/sociedad/aemet-fecha-vuelta...
1,https://www.abc.es/internacional/asesinan-deca...,https://www.abc.es/internacional/asesinan-deca...
2,https://www.abc.es/gente/nueva-vida-errante-to...,https://www.abc.es/gente/nueva-vida-errante-to...
3,https://www.abc.es/espana/madrid/fiestas-san-c...,https://www.abc.es/espana/madrid/fiestas-san-c...
4,https://www.abc.es/viajar/destinos/espana/carr...,https://www.abc.es/viajar/destinos/espana/carr...
...,...,...
291,https://www.autocasion.com/,NaN
292,https://www.pisos.com/,NaN
293,https://www.unoauto.com/,NaN
294,https://www.infoempleo.com/,NaN


In [23]:
import sqlite3
PATH_DATA = os.path.join("..", "data")
DB_NAME_NEWS = os.path.join(PATH_DATA, "news_db.sqlite3")
def read_stored_news(where_params):
    with sqlite3.connect(DB_NAME_NEWS, timeout=10) as conn:
        cursor = conn.cursor()
        create_news_table(conn, 
                        cursor)
        if not isinstance(where_params, (tuple, list)):
            where_params = (where_params, )
        query_str = """
            SELECT 
                url
            FROM 
                News
            WHERE
                source = ?;
            """
        output = cursor.execute(query_str, where_params)
        conn.commit()
    return output

def create_news_table(conn, 
                      cursor):
    query_str = """
        CREATE TABLE IF NOT EXISTS News (
            title TEXT NOT NULL,
            article TEXT NOT NULL,
            source TEXT,
            country TEXT,
            creationDate TEXT NOT NULL,
            updateDate TEXT,
            url TEXT PRIMARY KEY NOT NULL,
            image_url TEXT,
            tags TEXT,
            insertDate Text NOT NULL,
            changeDate Text,
            number_tokens Integer
        )
            ;
        """
    cursor.execute(query_str)
    conn.commit()
media = "https://okdiario.com"
media = "https://www.lavozdegalicia.es/"
media = "https://www.huffingtonpost.es/"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
response = requests.get(media, 
                        headers=HEADERS, 
                        timeout=10)

parsed_hmtl = BeautifulSoup(response.content, 
                            "html.parser")
tags_with_url = parsed_hmtl.find_all("a", 
                                     href=re.compile(r"^(?:https:\/\/)?|^\/{1}[^\/].*|^www[.].*"))
                                     # href=re.compile("https?:.*"))
news_urls = [x.attrs["href"] for x in tags_with_url if x.attrs.get("href", False)]
urls = []
for url in news_urls:
    if url.startswith("//"):
        url = "https:" + url
    elif url.startswith("/"):
        url = media[:-1] + url
    elif url.startswith("www.") or not url.startswith("https://"):
        url = "https://" + url
    urls.append(url)



In [154]:
select_urls = pd.DataFrame(read_stored_news(media), 
                           columns=["in_store"])
                     
extracted_urls = pd.DataFrame(urls, 
                              columns=["in_extraction"]
                             ).drop_duplicates()
df_novels = select_urls.merge(extracted_urls, 
                              left_on="in_store", 
                              right_on="in_extraction", 
                              how="right")

print(df_novels.shape)
df_novels.head(10)

(149, 2)


,in_store,in_extraction
0,NaN,https://www.huffingtonpost.es/tiempo/dana-espa...
1,NaN,https://www.huffingtonpost.es/sociedad/facua-r...
2,https://www.huffingtonpost.es/sociedad/la-zona...,https://www.huffingtonpost.es/sociedad/la-zona...
3,NaN,https://www.huffingtonpost.es/tiempo/la-aemet-...
4,NaN,https://www.huffingtonpost.es/global/guerra-uc...
5,NaN,https://www.huffingtonpost.es/virales/ayuso-vu...
6,https://www.huffingtonpost.es/economia/estas-s...,https://www.huffingtonpost.es/economia/estas-s...
7,NaN,https://www.huffingtonpost.es/virales/la-polic...
8,NaN,https://www.huffingtonpost.es/virales/una-espa...
9,NaN,https://www.huffingtonpost.es/life/influencers...


In [161]:
are_new_urls = df_novels.in_store.isnull()
df_novels_extraction = df_novels[are_new_urls]
print("All nulls in_store column:", df_novels_extraction.in_store.isnull().all())
print(f"Read; extracted;to_process: {len(select_urls)}, {len(extracted_urls)}, {len(df_novels_extraction)} ({are_new_urls.sum()})")

All nulls in_store column: True
Read; extracted;to_process: 132, 149, 114 (114)


In [166]:
df_novels_extraction.in_extraction.isnull().sum(), \
df_novels_extraction.in_extraction.duplicated().sum()

(0, 0)

In [157]:
for x, y in df_novels_nodups.values:
    print("In store:", x)
    print("In extracted:", y, end="\n\n")

In store: nan
In extracted: https://www.huffingtonpost.es/tiempo/dana-espana-ultima-hora-lluvias-posibles-inundaciones-carreteras-cortadas-hoy-directo.html

In store: nan
In extracted: https://www.huffingtonpost.es/sociedad/facua-responde-todos-piden-denuncie-notificacion-enviada-proteccion-civil-madrid.html

In store: nan
In extracted: https://www.huffingtonpost.es/tiempo/la-aemet-da-hora-clave-dana.html

In store: nan
In extracted: https://www.huffingtonpost.es/global/guerra-ucrania-rusia-directo-ultimas-noticias-4-septiembrebr.html

In store: nan
In extracted: https://www.huffingtonpost.es/virales/ayuso-vuelve-hablar-caso-rubiales-resume-frase-trae.html

In store: nan
In extracted: https://www.huffingtonpost.es/virales/la-policia-nacional-recomienda-esto-mas-entrar-casa-evitar-sustos-sencillo.html

In store: nan
In extracted: https://www.huffingtonpost.es/virales/una-espanola-comprueba-cuesta-aceite-oliva-alemania-ver-irlanda-sorpresa.html

In store: nan
In extracted: https://www.hu

In [167]:
df_validation = df_novels_nodups.merge(extracted_urls, 
                                  on="in_extraction", 
                                  how="right")
df_validation.drop_duplicates().shape

(149, 2)

Take all topics from header

In [172]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
url = "https://www.lavanguardia.com"
url = "https://www.abc.es/"
url = "https://www.elpais.com/"
response = requests.get(url, headers=headers)

parsed_hmtl = BeautifulSoup(response.content, "html.parser")

In [173]:
links = [x.attrs.get("href", None) for x in parsed_hmtl.body.header.find_all("a")]
links

['https://elpais.com',
 'https://elpais.com/america/',
 'https://elpais.com/mexico/',
 'https://elpais.com/america-colombia/',
 'https://elpais.com/chile/',
 'https://elpais.com/argentina/',
 'https://english.elpais.com',
 'https://elpais.com',
 'https://elpais.com/suscripciones/#/campaign#?prod=SUSDIG&o=boton_cab&prm=suscrip_cabecera_el-pais&backURL=https%3A%2F%2Felpais.com',
 'https://elpais.com/subscriptions/#/sign-in?prod=REG&o=CABEP&prm=login_cabecera_el-pais&backURL=https%3A%2F%2Felpais.com',
 '/internacional/',
 '/opinion/',
 '/espana/',
 '/economia/',
 '/sociedad/',
 '/educacion/',
 'https://elpais.com/clima-y-medio-ambiente/',
 'https://elpais.com/ciencia/',
 'https://elpais.com/salud-y-bienestar/',
 '/tecnologia/',
 '/cultura/',
 'https://elpais.com/babelia/',
 '/deportes/',
 '/television/',
 'https://elpais.com/gente/',
 'https://elpais.com/eps/',
 'https://elpais.com/suscripciones/#/campaign#?prod=SUSDIGCRART&o=boton_cab&backURL=https%3A%2F%2Felpais.com']

In [27]:
links_serie = pd.Series(links).dropna()
nodes = links_serie.str.replace("https://www.abc.es", "", regex=True)
nodes.head(20)

0     https://www.lavanguardia.com/internacional/202...
1     https://www.lavanguardia.com/internacional/202...
2     https://www.lavanguardia.com/local/girona/2023...
3     https://www.lavanguardia.com/local/navarra/202...
4     https://www.lavanguardia.com/television/202308...
5     https://www.lavanguardia.com/gente/20230804/91...
6     https://www.lavanguardia.com/politica/20230804...
7     https://www.lavanguardia.com/gente/20230804/91...
8     https://www.lavanguardia.com/gente/20230804/91...
9     https://www.lavanguardia.com/cribeo/20230804/9...
10    https://www.lavanguardia.com/politica/20230804...
11    https://www.lavanguardia.com/natural/20230804/...
12    https://www.lavanguardia.com/deportes/futbol/2...
13    https://www.lavanguardia.com/television/202308...
14    https://www.lavanguardia.com/television/202308...
15    https://www.lavanguardia.com/politica/20230804...
16    https://www.lavanguardia.com/television/202308...
17    https://www.lavanguardia.com/cribeo/202308

In [22]:
nodes_split = nodes.str.split("/")
nodes_split_clean = nodes_split.apply(lambda x: [y for y in x if y])
nodes_split_clean

0      [https:, www.lavanguardia.com, internacional, ...
1      [https:, www.lavanguardia.com, internacional, ...
2      [https:, www.lavanguardia.com, local, girona, ...
3      [https:, www.lavanguardia.com, local, navarra,...
4      [https:, www.lavanguardia.com, television, 202...
                             ...                        
108                                           [economia]
109                                              [local]
110                                              [gente]
111                                            [cultura]
112                                            [sucesos]
Length: 107, dtype: object

In [23]:
nodes_split_clean_filter = nodes_split_clean.apply(lambda x: True if len(x) == 1 else False)
nodes_split_clean_filter

0      False
1      False
2      False
3      False
4      False
       ...  
108     True
109     True
110     True
111     True
112     True
Length: 107, dtype: bool

In [24]:
links_valid = links_serie[nodes_split_clean_filter]
links_valid

102         /alminuto
103    /internacional
104         /politica
105          /opinion
107         /deportes
108         /economia
109            /local
110            /gente
111          /cultura
112          /sucesos
dtype: object

In [39]:
nodes_split_clean.str.len().eq(1)

0      6
1      6
2      7
3      7
4      6
      ..
108    1
109    1
110    1
111    1
112    1
Length: 107, dtype: int64

In [18]:
x = "https://www.google.es"
x.replace("google", "amazon")

'https://www.amazon.es'

In [19]:
x

'https://www.google.es'

In [9]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
url = "https://www.lavanguardia.com"
#url = "https://www.abc.es/"
url = "https://okdiario.com/"
url = "https://lavozdegalicia.es"
url = "https://www.huffingtonpost.es"
response = requests.get(url, headers=headers)

parsed_hmtl = BeautifulSoup(response.content, "html.parser")
links = [x.attrs.get("href", None) for x in parsed_hmtl.body.find_all("a")]

links_serie = pd.Series(links).dropna()
nodes = links_serie.str.replace(url, "", regex=True)

nodes_split = nodes.str.split("/")
nodes_split_clean = nodes_split.apply(lambda x: [elem for elem in x if elem])

nodes_split_clean_filter = nodes_split_clean.str.len().eq(1)

links_valid = links_serie[nodes_split_clean_filter]
links_valid_complete = [url + x if not x.startswith(url) else x for x in links_valid ]
pd.Series(links_valid_complete).unique().tolist()

['https://www.huffingtonpost.es/politica',
 'https://www.huffingtonpost.es/global',
 'https://www.huffingtonpost.es/virales',
 'https://www.huffingtonpost.es/life',
 'https://www.huffingtonpost.es/videos',
 'https://www.huffingtonpost.es/ultimas-noticias',
 'https://www.huffingtonpost.es/tiempo',
 'https://www.huffingtonpost.es/sociedad',
 'https://www.huffingtonpost.es/economia',
 'https://www.huffingtonpost.es/opinion',
 'https://www.huffingtonpost.es/lgtbi',
 'https://www.huffingtonpost.es/planeta',
 'https://www.huffingtonpost.es/tecnologia',
 'https://www.huffingtonpost.es/deporte',
 'https://www.huffingtonpost.es/medios',
 'https://www.huffingtonpost.es/loterias',
 'https://www.huffingtonpost.es/pillalo',
 'https://www.huffingtonpost.esjavascript:Didomi.preferences.show()']

In [15]:
os.listdir()

['.ipynb_checkpoints',
 'Analysis of audited media ojdinteractiva.ipynb',
 'data',
 'db.sqlite3',
 'development and utest_generalizable_scrapper.ipynb',
 'Errors log.txt',
 'Extraction of audited media scimago media.ipynb',
 'Extraction of audited media.ipynb',
 'News scrapper and loader_general http extraction.py',
 'obsolete',
 'registro de medios sin json disponible en codigo html.txt',
 'Selenium_News scrapper and loader_general http extraction.py',
 'statistics_2023-08-04.csv',
 'statistics_2023-08-07.csv',
 'Unit tests.ipynb',
 'URL cross-comparation.ipynb',
 'utilities.py',
 'Wrappers.ipynb',
 '__pycache__']

In [14]:
os.path.join(".", "data", "statistics_2023-08-07.csv")

'.\\data\\statistics_2023-08-07.csv'

In [17]:
os.path.exists(os.path.join(".", "statistics_2023-08-07.csv"))

True

In [7]:
parsed_hmtl.body.find_all("a")

[<a aria-label="Volver a portada" class="logo flex sz-100" href="/">
 <span class="d-none">La Voz de Galicia</span>
 <!-- strong>Actualizado:6:46:51 PM</strong-->
 </a>,
 <a href="/">PORTADA</a>,
 <a href="/galicia/">GALICIA</a>,
 <a href="/localidades/">LOCAL</a>,
 <a href="/economia/">ECONOMÍA</a>,
 <a href="/espana/">ESPAÑA</a>,
 <a href="/internacional/">INTERNACIONAL</a>,
 <a href="/opinion/">OPINIÓN</a>,
 <a href="/deportes/">DEPORTES</a>,
 <a href="/sociedad/">SOCIEDAD</a>,
 <a href="/cultura/">CULTURA</a>,
 <a href="/somosagro/">AGRO</a>,
 <a href="/somosmar/">MAR</a>,
 <a href="/alsol/">AL SOL</a>,
 <a href="/coruna/">A CORUÑA</a>,
 <a href="/amarina/">A MARIÑA</a>,
 <a href="/arousa/">AROUSA</a>,
 <a href="/barbanza/">BARBANZA</a>,
 <a href="/carballo/">CARBALLO</a>,
 <a href="/deza/">DEZA</a>,
 <a href="/ferrol/">FERROL</a>,
 <a href="/lemos/">LEMOS</a>,
 <a href="/lugo/">LUGO</a>,
 <a href="/ourense/">OURENSE</a>,
 <a href="/pontevedra/">PONTEVEDRA</a>,
 <a href="/santiago/

In [109]:
for url_section in links_valid_complete:
    response = requests.get(url_section, headers=headers)

    parsed_hmtl = BeautifulSoup(response.content, "html.parser")
    links = [x.attrs.get("href", None) for x in parsed_hmtl.body.header.find_all("a")]

    links_serie = pd.Series(links).dropna()
    nodes = links_serie.str.replace(url_section, "", regex=True)

    nodes_split = nodes.str.split("/")
    nodes_split_clean = nodes_split.apply(lambda x: [elem for elem in x if elem])

    nodes_split_clean_filter = nodes_split_clean.str.len().eq(2)

    links_valid = links_serie[nodes_split_clean_filter]
    
    links_valid_complete2 = []
    for x in links_valid:
        first_section_node = [x for x in url_section.split("/") if x][-1]
        if first_section_node != x:
            if first_section_node in x:
                if url_section in x:
                    links_valid_complete2.append(x)
                else:
                    if url_section.endswith("/"):
                        links_valid_complete2.append(url_section + x)
                    else:
                        links_valid_complete2.append(url_section + "/" + x)
    break

In [129]:
nodes.tail(10).iloc[0]

'https://okdiario.com/economia/error-que-cometemos-pedir-copia-tarjeta-banco-espana-lanza-alerta-11351248'

In [124]:
nodes_split_clean

1                                 [https:, okdiario.com]
2                                                     []
3                         [https:, okdiario.com, espana]
4                         [https:, okdiario.com, madrid]
5                       [https:, okdiario.com, cataluna]
                             ...                        
130    [https:, okdiario.com, sociedad, adios-hacer-o...
131    [https:, okdiario.com, economia, cambio-drasti...
132                                                   []
133                                   [ultimas-noticias]
134                                                   []
Length: 131, dtype: object

In [120]:
nodes_split_clean

1                                 [https:, okdiario.com]
2                                                     []
3                         [https:, okdiario.com, espana]
4                         [https:, okdiario.com, madrid]
5                       [https:, okdiario.com, cataluna]
                             ...                        
130    [https:, okdiario.com, sociedad, adios-hacer-o...
131    [https:, okdiario.com, economia, cambio-drasti...
132                                                   []
133                                   [ultimas-noticias]
134                                                   []
Length: 131, dtype: object

In [119]:
nodes.str.contains("deportes/futbol").sum()

1

In [32]:
daemon_thread.isDaemon = False

Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.


In [21]:
import threading
import time

def daemon_function():
    while True:
        print("Daemon thread is running.")
        time.sleep(1)

# Create a daemon thread
daemon_thread = threading.Thread(target=daemon_function)
daemon_thread.daemon = True  # Set the thread as daemon

# Start the daemon thread
daemon_thread.start()

# Main thread waits for a few seconds
time.sleep(5)
print("Main thread finished.")


Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Main thread finished.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.


In [22]:
import threading
import time

def producer(queue):
    for i in range(5):
        print(f"Producing item {i}")
        queue.put(i)
        time.sleep(1)

def consumer(queue):
    while True:
        item = queue.get()
        if item is None:
            break
        print(f"Consuming item {item}")
        time.sleep(2)

# Create a queue for communication
queue = queue.Queue()

# Create producer and consumer threads
producer_thread = threading.Thread(target=producer, args=(queue,))
consumer_thread = threading.Thread(target=consumer, args=(queue,))

# Start threads
producer_thread.start()
consumer_thread.start()

# Wait for producer to finish
producer_thread.join()

# Signal consumer to stop
queue.put(None)
consumer_thread.join()

print("Main thread finished.")

NameError: name 'queue' is not defined

Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon threa

Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon threa

Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon threa

Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon threa

...

Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon threa

Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon threa

Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon threa

Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon threa

Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon thread is running.
Daemon threa

In [3]:
os.listdir()

['.ipynb_checkpoints',
 'Analysis of audited media ojdinteractiva.ipynb',
 'data',
 'db.sqlite3',
 'development and utest_generalizable_scrapper.ipynb',
 'Errors log.txt',
 'Extraction of audited media scimago media.ipynb',
 'Extraction of audited media.ipynb',
 'News scrapper and loader_general http extraction.py',
 'obsolete',
 'registro de medios sin json disponible en codigo html.txt',
 'Selenium_News scrapper and loader_general http extraction.py',
 'statistics_2023-08-04.csv',
 'statistics_2023-08-07.csv',
 'Unit tests.ipynb',
 'URL cross-comparation.ipynb',
 'utilities.py',
 'Wrappers.ipynb',
 '__pycache__']

In [22]:
with open("data/spain_media name_to_url.json", "r") as file, open("data/plain_spain_media name_to_url.txt", "w") as file_w:
    data = json.loads(file.read())
    data_plain = "\n".join([k.strip() + ";" + v for k, v in data.items()])
    file_w.write(data_plain)

In [17]:
print(data_plain)

El País;elpais.com
ABC;abc.es
El Español;elespanol.com
El Mundo;elmundo.es
La Vanguardia;lavanguardia.com
El Periódico de Catalunya;elperiodico.com
La Razón;larazon.es
20 Minutos;20minutos.es
ABC de Sevilla;sevilla.abc.es/
Europa Press;europapress.es
elDiario.es;eldiario.es
La Voz de Galicia;lavozdegalicia.es
El Correo Español;elcorreo.com
Libertad Digital;libertaddigital.com
SUR;diariosur.es
Diario de Sevilla;diariodesevilla.es
Crónica global;cronicaglobal.elespanol.com
El Diario Vasco;diariovasco.com
La Verdad;laverdad.es
Levante;levante-emv.com
El Confidencial;elconfidencial.com
Las Provincias;lasprovincias.es
Heraldo de Aragón;heraldo.es
OK Diario;okdiario.com
Ideal;ideal.es
Huffington Post España;huffingtonpost.es
Faro de Vigo;farodevigo.es
Canarias 7;canarias7.es
El Periódico de España;epe.es
La Información;lainformacion.com
Periodista Digital;periodistadigital.com
Publico;publico.es
DEIA;deia.eus
El Comercio;elcomercio.es
Diario de Mallorca;diariodemallorca.es
Hoy de Badajoz;hoy

In [24]:
import time

In [34]:
class StatisticsReporter():
    def __init__(self) -> None:
        self.time_start = 0.0
    
    def restart_time(self):
        self.time_start = time.perf_counter()

    def write_extraction_stats(self):
        return str(time.perf_counter() - self.time_start)

In [37]:
reporter = StatisticsReporter()

In [38]:
reporter.restart_time()

time.sleep(2)

reporter.write_extraction_stats()

'2.0007124000001113'

In [45]:
i = 0
while True:
    time.sleep(0.2)
    print(f"2 x {i} :", i * 2)
    i += 1

2 x 0 : 0
2 x 1 : 2
2 x 2 : 4
2 x 3 : 6
2 x 4 : 8
2 x 5 : 10
2 x 6 : 12
2 x 7 : 14
2 x 8 : 16
2 x 9 : 18
2 x 10 : 20
2 x 11 : 22
2 x 12 : 24
2 x 13 : 26
2 x 14 : 28
2 x 15 : 30
2 x 16 : 32
2 x 17 : 34
2 x 18 : 36
2 x 19 : 38
2 x 20 : 40
2 x 21 : 42
2 x 22 : 44
2 x 23 : 46
2 x 24 : 48
2 x 25 : 50
2 x 26 : 52
2 x 27 : 54
2 x 28 : 56
2 x 29 : 58
2 x 30 : 60
2 x 31 : 62
2 x 32 : 64
2 x 33 : 66
2 x 34 : 68
2 x 35 : 70
2 x 36 : 72
2 x 37 : 74
2 x 38 : 76
2 x 39 : 78
2 x 40 : 80
2 x 41 : 82
2 x 42 : 84
2 x 43 : 86
2 x 44 : 88
2 x 45 : 90
2 x 46 : 92
2 x 47 : 94
2 x 48 : 96
2 x 49 : 98
2 x 50 : 100
2 x 51 : 102
2 x 52 : 104
2 x 53 : 106
2 x 54 : 108
2 x 55 : 110
2 x 56 : 112
2 x 57 : 114
2 x 58 : 116
2 x 59 : 118
2 x 60 : 120
2 x 61 : 122
2 x 62 : 124
2 x 63 : 126
2 x 64 : 128
2 x 65 : 130
2 x 66 : 132
2 x 67 : 134
2 x 68 : 136
2 x 69 : 138
2 x 70 : 140
2 x 71 : 142
2 x 72 : 144
2 x 73 : 146
2 x 74 : 148
2 x 75 : 150
2 x 76 : 152
2 x 77 : 154
2 x 78 : 156
2 x 79 : 158
2 x 80 : 160
2 x 81 : 162

KeyboardInterrupt: 

In [28]:
url = "https://www.france24.com/en/live-news/20230818-drug-war-bloodbath-shakes-marseille-as-rival-gangs-shoot-it-out-1"
url = "https://www.huffingtonpost.es/politica/campana-feijoo-pese-ganar-14700-euros-mes-politica-atesorar-patrimonio.html"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=HEADERS)

In [29]:
response.status_code

200

In [30]:
parsed_html = BeautifulSoup(response.content, "html.parser")

In [60]:
from datetime import timedelta

In [37]:
time_tags = parsed_html.select("time")
timestamps = {t.get("pubdate", f"date_{i}"): t.attrs["datetime"] for i, t in enumerate(time_tags)}
timestamps

{'date_0': '2023-08-18T12:24:38+02:00', 'date_1': '2023-08-18'}

In [155]:
today_iso = datetime.today().isoformat(sep=" ")
today_iso, type(today_iso), str(datetime.fromisoformat(today_iso))

('2023-08-18 19:04:39.797575', str, '2023-08-18 19:04:39.797575')

In [134]:
timestamp = datetime.fromisoformat(timestamps["date_0"]).strftime("%Y-%m-%d %H:%M:%S")
date = str(timestamp.date())
time = str(timestamp.time())
timestamp, type(timestamp), str(timestamp), date, time

(datetime.datetime(2023, 8, 18, 12, 24, 38, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))),
 datetime.datetime,
 '2023-08-18 12:24:38+02:00',
 '2023-08-18',
 '12:24:38')

In [154]:
today_iso >= timestamp.strftime("%Y-%m-%d %H:%M:%S")

True

In [152]:
timestamp, timestamp - timedelta(days=2)

(datetime.datetime(2023, 8, 18, 12, 24, 38, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))),
 datetime.datetime(2023, 8, 16, 12, 24, 38, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))))

### JSON FROM HTML

In [92]:
url1 = "https://www.france24.com/en/live-news/20230818-drug-war-bloodbath-shakes-marseille-as-rival-gangs-shoot-it-out-1"
url2 = "https://www.huffingtonpost.es/politica/campana-feijoo-pese-ganar-14700-euros-mes-politica-atesorar-patrimonio.html"
url3 = "https://www.lavanguardia.com/economia/20230819/9174627/alimentos-ponen-tension-objetivos-inflacion-zona-euro.html"
url4 = "https://www.xataka.com/basics/como-declarar-a-hacienda-las-inversiones-en-bitcoin-y-criptomonedas-con-beneficios"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
response = requests.get(url4, 
                        headers=HEADERS)
parsed_html = BeautifulSoup(response.content, 
                            "html.parser")

In [66]:
def find_news_body(html, data):
    is_article = False
    id_pub_date = False
    is_mod_date = False
    is_title = False
    data["body"] = None
    data["n_tokens"] = 0
    jsons = html.find_all("script", 
                          attrs={"type": re.compile("application[/]{1}ld[+]{1}json")})
    print(jsons)
    result = []
    for json_data_str in jsons:
        json_data = json.loads(json_data_str.get_text(), 
                               strict=False)
        if isinstance(json_data, list):
            if json_data:
                json_data = json_data[0]
            else:
                continue
        if not is_body and "articleBody" in json_data:
            body, n_tokens = ("body", 1) #get_body_summary(remove_body_tags(json_data["articleBody"]))
            data["body"] = body
            data["n_tokens"] = n_tokens
            #break
        elif not is_article and "@type" in json_data:
            type_value = json_data["@type"].lower()
            if "news" in type_value or  "article" in json_data["@type"]:
                is_article = True
        elif not is_pub_date and "datePublished" in json_data:
            data["creation_datetime"] = json_data["datePublished"]
            is_pub_date = True
        elif not is_mod_date and "dateModified" in json_data:
            data["creation_datetime"] = json_data["dateModified"]
            is_mod_date = True
        elif not is_title and "headline" in json_data:
            data["title"] = json_data["headline"]
            is_title = True
        elif not is_tags and ("keywords" in json_data or "tags" in json_data):
            for key in ["keywords", "tags"]:
                try:
                    data["tags"] = json_data[key]
                    is_tags = True
                    break
                except:
                    pass
        result.append(json_data)
    return data, result

In [67]:
data = {}
data, result = find_news_body(parsed_html, data)

[<script type="application/ld+json">{"@type":"NewsArticle","@context":"https:\/\/schema.org","headline":"Lo que dijo en campa\u00f1a Feij\u00f3o pese a ganar 14.700 euros al mes: \"No es posible estar en pol\u00edtica y atesorar un patrimonio\"","alternativeHeadline":"Su declaraciones de bienes presentada al Congreso desvela lo que cobra como senador y sus dos sobresueldos del partido","datePublished":"2023-08-18T12:24:38+0200","dateModified":"2023-08-18T16:25:29+0200","publisher":{"@type":"NewsMediaOrganization","name":"El HuffPost","url":"https:\/\/www.huffingtonpost.es","sameAs":["https:\/\/www.facebook.com\/ElHuffPost","https:\/\/twitter.com\/ElHuffPost","https:\/\/www.instagram.com\/elhuffpost"],"logo":{"@type":"ImageObject","@id":"#logoImage","url":"https:\/\/www.huffingtonpost.es\/images\/logoHUFF.png","height":"34","width":"304"}},"description":"El pasado mes de mayo, en plena campa\u00f1a por las elecciones municipales y auton\u00f3micas del 28-M, Alberto N\u00fa\u00f1ez Feij\

NameError: name 'is_body' is not defined

In [70]:
data

{'body': 'body', 'n_tokens': 1}

In [72]:
print(data)
print(len(result), len(result[0]))

for k1, vs1 in result[0].items():
    if isinstance(vs1, dict):
        print(k1, ":", sep="")
        for k2, vs2 in vs1.items():
            if isinstance(vs2, list):
                print("\t", k2, len(vs2))
            else:
                print("\t", k2, 1)
    else:
        print(k1, 1)

{'body': 'body', 'n_tokens': 1}
5 16
@type 1
@context 1
headline 1
alternativeHeadline 1
datePublished 1
dateModified 1
publisher:
	 @type 1
	 name 1
	 url 1
	 sameAs 3
	 logo 1
description 1
keywords 1
articleSection 1
articleBody 1
mainEntityOfPage:
	 @type 1
	 @id 1
author:
	 @type 1
	 name 1
	 url 1
image 1
video 1
audio 1


In [73]:
result[0]

{'@type': 'NewsArticle',
 '@context': 'https://schema.org',
 'headline': 'Lo que dijo en campaña Feijóo pese a ganar 14.700 euros al mes: "No es posible estar en política y atesorar un patrimonio"',
 'alternativeHeadline': 'Su declaraciones de bienes presentada al Congreso desvela lo que cobra como senador y sus dos sobresueldos del partido',
 'datePublished': '2023-08-18T12:24:38+0200',
 'dateModified': '2023-08-18T16:25:29+0200',
 'publisher': {'@type': 'NewsMediaOrganization',
  'name': 'El HuffPost',
  'url': 'https://www.huffingtonpost.es',
  'sameAs': ['https://www.facebook.com/ElHuffPost',
   'https://twitter.com/ElHuffPost',
   'https://www.instagram.com/elhuffpost'],
  'logo': {'@type': 'ImageObject',
   '@id': '#logoImage',
   'url': 'https://www.huffingtonpost.es/images/logoHUFF.png',
   'height': '34',
   'width': '304'}},
 'description': 'El pasado mes de mayo, en plena campaña por las elecciones municipales y autonómicas del 28-M, Alberto Núñez Feijóo hizo un ejercicio de

In [162]:
def find_news_body_and_others(html, data):
    extraction_completed = False
    is_body = False
    is_article = False
    is_pub_date = False
    is_mod_date = False
    is_title = False
    is_tags = False
    data["body"] = None
    data["n_tokens"] = 0
    jsons = html.find_all("script", 
                          attrs={"type": re.compile("application[/]{1}ld[+]{1}json")})
    for json_data_str in jsons:
        if extraction_completed:
            break
        json_data = json.loads(json_data_str.get_text(), 
                               strict=False)
        if isinstance(json_data, list):
            if json_data:
                json_data = json_data[0]
        for k, v in json_data.items():
            print(k)
            if "@type" in k:
                data["type"] = v
            if not is_body and "articleBody" in k:
                body, n_tokens = get_body_summary(remove_body_tags(json_data["articleBody"]))
                data["body"] = body
                data["n_tokens"] = n_tokens
                is_body = True
            elif not is_article and "@type" in k:
                type_value = json_data["@type"].lower()
                if "news" in type_value or  "article" in json_data["@type"]:
                    is_article = True
            elif not is_pub_date and "datePublished" in k:
                data["creation_datetime"] = json_data["datePublished"]
                is_pub_date = True
            elif not is_mod_date and "dateModified" in k:
                data["modified_datetime"] = json_data["dateModified"]
                is_mod_date = True
            elif not is_title and "headline" in k:
                data["title"] = json_data["headline"]
                is_title = True
            elif not is_tags and ("keywords" in k or "tags" in k):
                for sub_tag_key in ["keywords", "tags"]:
                    try:
                        data["tags"] = json_data[sub_tag_key]
                        is_tags = True
                        break
                    except:
                        pass
            if all((is_body, is_title, is_article, is_pub_date, is_mod_date, is_tags)):
                extraction_completed = True
                break
    return data

In [166]:
find_news_body_and_others(parsed_html, data)

@context
@type
mainEntityOfPage
name
headline
datePublished
dateModified
description
publisher
image
author
url
thumbnailUrl
articleSection
creator
keywords


{'body': None,
 'n_tokens': 0,
 'title': 'Cómo declarar a Hacienda las inversiones en bitcoin y criptomonedas con beneficios',
 'creation_datetime': '2023-02-09T14:33:26Z',
 'tags': 'Bitcoin, Xataka Basics, Criptomonedas',
 'modified_datetime': '2023-02-09T14:33:26Z',
 'type': 'Article'}

In [164]:
x = {"a": True, "b": True, "c": True}
all(x.values())

True

In [165]:
parsed_html.select("html head meta[property],[name]")

[<meta content="width=device-width, initial-scale=1.0" name="viewport"/>,
 <meta content="El Bitcoin y las criptomonedas llevan varias semanas en boca de todos, haciendo que tanto expertos en la tecnología como personas de a pie empecemos a..." name="description"/>,
 <meta content="Bitcoin, Xataka Basics, Criptomonedas" name="news_keywords"/>,
 <meta content="noodp" name="robots"/>,
 <meta content="100000716994885" property="fb:admins"/>,
 <meta content="41267802635" property="fb:pages"/>,
 <meta content="355823546895" property="fb:app_id"/>,
 <meta content="Xataka" name="application-name"/>,
 <meta content="Gadgets y tecnología: últimas tecnologías en electrónica de consumo - XATAKA" name="msapplication-tooltip"/>,
 <meta content="https://www.xataka.com" name="msapplication-starturl"/>,
 <meta content="yes" name="apple-mobile-web-app-capable"/>,
 <meta content="name=Lo mejor;action-uri=https://www.xataka.com/lomejor;icon-uri=https://img.weblogssl.com/css/xataka/p/v6/images/pin-bg-lome

In [141]:
parsed_html.select("html head meta[name]")

[<meta content="width=device-width, initial-scale=1.0" name="viewport"/>,
 <meta content="El Bitcoin y las criptomonedas llevan varias semanas en boca de todos, haciendo que tanto expertos en la tecnología como personas de a pie empecemos a..." name="description"/>,
 <meta content="Bitcoin, Xataka Basics, Criptomonedas" name="news_keywords"/>,
 <meta content="noodp" name="robots"/>,
 <meta content="Xataka" name="application-name"/>,
 <meta content="Gadgets y tecnología: últimas tecnologías en electrónica de consumo - XATAKA" name="msapplication-tooltip"/>,
 <meta content="https://www.xataka.com" name="msapplication-starturl"/>,
 <meta content="yes" name="apple-mobile-web-app-capable"/>,
 <meta content="name=Lo mejor;action-uri=https://www.xataka.com/lomejor;icon-uri=https://img.weblogssl.com/css/xataka/p/v6/images/pin-bg-lomejor-icon.ico" name="msapplication-task"/>,
 <meta content="Yúbal Fernández" name="DC.Creator"/>,
 <meta content="2018-02-12" name="DC.Date"/>,
 <meta content="2023

In [151]:
parsed_html.html.head.find_all("meta")

[<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>,
 <meta content="width=device-width, initial-scale=1.0" name="viewport"/>,
 <meta content="El Bitcoin y las criptomonedas llevan varias semanas en boca de todos, haciendo que tanto expertos en la tecnología como personas de a pie empecemos a..." name="description"/>,
 <meta content="Bitcoin, Xataka Basics, Criptomonedas" name="news_keywords"/>,
 <meta content="noodp" name="robots"/>,
 <meta content="100000716994885" property="fb:admins"/>,
 <meta content="41267802635" property="fb:pages"/>,
 <meta content="355823546895" property="fb:app_id"/>,
 <meta content="Xataka" name="application-name"/>,
 <meta content="Gadgets y tecnología: últimas tecnologías en electrónica de consumo - XATAKA" name="msapplication-tooltip"/>,
 <meta content="https://www.xataka.com" name="msapplication-starturl"/>,
 <meta content="yes" name="apple-mobile-web-app-capable"/>,
 <meta content="name=Lo mejor;action-uri=https://www.xataka.com/lomejo

### Investigate ways to differenciate media sections to news articles

In [7]:
url1 = "https://www.france24.com/en/live-news/20230818-drug-war-bloodbath-shakes-marseille-as-rival-gangs-shoot-it-out-1"
url2 = "https://www.huffingtonpost.es/politica/campana-feijoo-pese-ganar-14700-euros-mes-politica-atesorar-patrimonio.html"
url3 = "https://www.lavanguardia.com/economia/20230819/9174627/alimentos-ponen-tension-objetivos-inflacion-zona-euro.html"
url4 = "https://www.xataka.com/basics/como-declarar-a-hacienda-las-inversiones-en-bitcoin-y-criptomonedas-con-beneficios"
url5 = "https://www.elconfidencial.com/"
url6 = "https://www.cronicaglobal.elespanol.com/".replace("www.", "")
url7 = "https://www.elespanol.com/"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
response = requests.get(url7, 
                        headers=HEADERS)
parsed_html = BeautifulSoup(response.content, 
                            "html.parser")
hrefs = parsed_html.html.body.find_all("a", href=re.compile(".*"))
len(hrefs)

589

In [8]:
hrefs

[<a href="https://www.elespanol.com/opinion/editoriales/20230822/machismo-rubiales-convierte-posicion-insostenible/788561138_14.html">El Rugido</a>,
 <a href="https://www.elespanol.com/ciencia/salud/20230822/suben-contagios-vuelven-mascarillas-hospitales-pasando-covid/788421503_0.html">Qué pasa con la COVID</a>,
 <a href="https://www.elespanol.com/reportajes/20230822/eloy-asesino-exmilitar-first-dates-mata-punaladas-hombre-bar-cordoba/788421541_0.html">Eloy, el asesino exmilitar de First Dates</a>,
 <a href="https://www.elespanol.com/deportes/futbol/20230821/apoteosis-futbol-femenino-mundial-marca-hito-sociedad-espanola/788171293_0.html">Apoteosis del fútbol femenino</a>,
 <a href="https://www.elespanol.com/deportes/futbol/20230820/madre-oculto-olga-carmona-muerte-padre-entero-volaba-sidney/788171310_0.html">La madre ocultó a Olga Carmona la muerte de su padre</a>,
 <a href="https://www.elespanol.com/deportes/futbol/20230821/reacciones-beso-rubiales-jenni-hermoso-en-directo-muere-padre

In [107]:
def read_json_media_urls_file(file_path):
    with open(file_path, "r") as file:
        return json.load(file)
def read_plain_media_urls_file(file_path):
    with open(file_path, "r") as file:
        return file.readlines()

In [21]:
%timeit re.compile("(?:log|in|out)").search("/logout#analytics-cabecera-comprimida:mi-cuent")
%timeit "logout" in "/logout#analytics-cabecera-comprimida:mi-cuent" or "login" in "/logout#analytics-cabecera-comprimida:mi-cuent" or "log" in "/logout#analytics-cabecera-comprimida:mi-cuent"

653 ns ± 8.21 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
39.6 ns ± 0.347 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [58]:
with open("data\SCImago Media Ranking - Spain - Spanish.xlsx", "r") as file:
    print(file.readlines())

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe6 in position 10: invalid continuation byte

In [437]:
media_ranks = pd.read_excel("data\SCImago Media Ranking - Spain - Spanish_20082023.xlsx").sort_values("Global_rank")
media_ranks["Media"] = media_ranks["Media"].str.strip()
media_ranks["Domain"] = media_ranks["Domain"].str.strip()
media_ranks[["Media", "Domain"]].head(50).to_csv("./data/spain_media name_to_url.txt", 
                                        index=False, 
                                        header=False,
                                        sep=";")

AttributeError: module 'pandas' has no attribute 'to_csv'

In [102]:
(media_ranks["Overall"].diff(-1) > 2=).sum()

4

In [105]:
media_ranks.head(50)

,Media,Domain,Country,Region,Language,Global_rank,Overall
213,El País,elpais.com,Spain,Western Europe,Spanish,7,84.50
212,ABC,abc.es,Spain,Western Europe,Spanish,28,78.25
210,La Vanguardia,lavanguardia.com,Spain,Western Europe,Spanish/Catalan,43,76.75
211,El Español,elespanol.com,Spain,Western Europe,Spanish,43,76.75
209,El Mundo,elmundo.es,Spain,Western Europe,Spanish,60,74.75
208,El Periódico de Catalunya,elperiodico.com,Spain,Western Europe,Spanish,89,72.75
206,La Razón,larazon.es,Spain,Western Europe,Spanish,115,71.25
207,Europa Press,europapress.es,Spain,Western Europe,Spanish,115,71.25
205,20 Minutos,20minutos.es,Spain,Western Europe,Spanish,124,70.75
204,El Confidencial,elconfidencial.com,Spain,Western Europe,Spanish,151,69.25


In [111]:
media_url

'https://www.20minutos.es\n'

In [149]:
media_url.replace("www.", "")

'https://diariocordoba.com'

In [696]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}

file_name = "spain_media name_to_url.txt"
file_path = os.path.join(".", "data", file_name)
if os.path.exists(file_path):
    print("Reading file of urls of regions...")
    name_to_media_urls = read_plain_media_urls_file(file_path)
    all_sections = []
skip_head = True
for media_line in name_to_media_urls:
    if skip_head:
        skip_head = False
        continue
    _, media_url = media_line.split(";")
    media_url = "https://www." + media_url.strip()
    try:
        response = requests.get(media_url, 
                                headers=HEADERS, 
                                timeout=10)  # Set an appropriate timeout value (in seconds)
    except requests.exceptions.Timeout:
        # Handle the timeout exception
        print("The request timed out.")
        #ErrorReporter(f"Get request timeout exception at {media_url}")
    except requests.exceptions.RequestException as e:
        # Handle other request exceptions
        print(f"An error occurred: {str(e)}")
        response = requests.get(media_url.replace("www.", ""), 
                                headers=HEADERS, 
                                timeout=10)  # Set an appropriate timeout value (in seconds)
        #ErrorReporter(f"Get request general exception, {e}, at {media_url}")

    parsed_hmtl = BeautifulSoup(response.content, "html.parser")
    #for x in parsed_hmtl.body.find_all("a"):
    #    x.attrs.get("href", None)
    try:
        links = [x.attrs.get("href", None) for x in parsed_hmtl.body.find_all("a")]
    except Exception as e:
        print(e)

    links_serie = pd.Series(links).dropna()
    nodes = links_serie.str.replace(media_url, "", regex=True)

    nodes_split = nodes.str.split("/")
    nodes_split_clean = nodes_split.apply(lambda x: [elem for elem in x if elem])

    nodes_split_clean_filter = nodes_split_clean.str.len().eq(1)

    valid_links = links_serie[nodes_split_clean_filter]
    valid_links_complete = [media_url + x if not x.startswith(media_url) else x for x in valid_links]
    valid_links_complete = []
    with open("Skipped urls.txt", "w") as file_skipped:
        for link_node in valid_links:
            link = link_node
            link_lower = link.lower()
            # Drop if 'garbage' node
            if link == media_url \
                or len(link.replace("/", "")) == 1 \
                or "#" in link_lower \
                or ":" in link_lower \
                or "@" in link_lower \
                or "php" in link_lower \
                or "javascript" in link_lower \
                or "mailto" in link_lower \
                or "cookie" in link_lower \
                or "feed" in link_lower \
                or "contact" in link_lower \
                or ("aviso" in link_lower and "legal" in link_lower) \
                or "inici" in link_lower \
                or "session" in link_lower \
                or "sesion" in link_lower \
                or "ads" in link_lower \
                or "publicidad" in link_lower \
                or "privacidad" in link_lower \
                or "condiciones" in link_lower \
                or "tags" in link_lower \
                or "premium" in link_lower \
                or "archiv" in link_lower \
                or "sorteo" in link_lower \
                or "loter" in link_lower \
                or "newsletter" in link_lower \
                or "podcast" in link_lower \
                or "logout" in link_lower \
                or "login" in link_lower \
                or link.endswith(".html") \
                or "notifica" in link_lower \
                or "push" in link_lower \
                or "servicio" in link_lower \
                or "esquela" in link_lower \
                or "defunci" in link_lower \
                or "favorito" in link_lower \
                or "suscri" in link_lower \
                or "subscrib" in link_lower \
                or "pasatiempo" in link_lower \
                or "compra" in link_lower \
                or "tienda" in link_lower \
                or "gráfico" in link_lower or "gráfico" in link_lower \
                or "foto" in link_lower \
                or "hemeroteca" in link_lower \
                or "video" in link_lower or "vídeo" in link_lower \
                or "play" in link_lower \
                or "patrocin" in link_lower \
                or "autor" in link_lower \
                or re.compile("\d{3,}").search(link):
                continue
            if not link.startswith(media_url):
                #if not link.startswith("https://www."):
                if not link.startswith("//www."): #
                    link_parts = link.split("/")
                    link_parts = [x for x in link_parts if x]
                    if len(link_parts) == 1:
                        if link.startswith("/"):
                            #print(f"1_1_{media_url} | {link}")
                            link = media_url + link
                        else:
                            #print(f"1_2_{media_url} | / | {link}")
                            link = media_url + "/" + link
                    else:
                        continue
                else:
                    #print(f"{link} skipped. find_media_section(). continue_1 ", file=file_skipped)
                    continue
            else:
                link_parts = link.split("/")
                link_parts = [x for x in link_parts if x][2:]
                if len(link_parts) > 1 or len(link_parts) == 0:
                    #print(link_pars)
                    continue
                    #if link.startswith("/"):
                    #    print(f"2_1_{media_url} | {link}")
                    #    link = media_url + link
                    #else:
                    #    print(f"2_2_{media_url} | / | {link}")
                    #    link = media_url + "/" + link
                else:
                    continue

            """link = link_node
            # base url is not in link node
            if not link_node.startswith(media_url):
                # www is in link node (not valid main url)
                if link_node.startswith("https://www."):
                    print(f"{link_node} skipped. find_media_section(). continue_1 ", file=file_skipped)
                    continue
                # Valid url node
                elif not link_node.startswith("/"):
                    link = media_url + "/" + link
                else:
                    link = media_url + link"""
        
            valid_links_complete.append(link)
        print(f"Valid links of {media_url.strip()}:", len(valid_links))
        all_sections.extend(valid_links_complete)
    valid_links_complete_unique = pd.Series(all_sections).unique().tolist()

Reading file of urls of regions...
Valid links of https://www.abc.es: 61
Valid links of https://www.lavanguardia.com: 81
Valid links of https://www.elespanol.com: 116
Valid links of https://www.elmundo.es: 85
Valid links of https://www.elperiodico.com: 16
Valid links of https://www.larazon.es: 63
Valid links of https://www.europapress.es: 107
Valid links of https://www.20minutos.es: 48
Valid links of https://www.elconfidencial.com: 30
Valid links of https://www.eldiario.es: 121
Valid links of https://www.sevilla.abc.es: 52
Valid links of https://www.lavozdegalicia.es: 104
Valid links of https://www.libertaddigital.com: 49
An error occurred: HTTPSConnectionPool(host='www.cronicaglobal.elespanol.com', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x00000297BA301E20>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Valid links of https://www.cronicaglobal.elespanol.com: 27
Valid links of h

In [697]:
len(all_sections), len(valid_links_complete_unique)

(1813, 711)

In [698]:
valid_links_complete_unique

['https://www.abc.es/opinion',
 'https://www.abc.es/elecciones-generales-23j/',
 'https://www.abc.es/espana',
 'https://www.abc.es/internacional',
 'https://www.abc.es/economia',
 'https://www.abc.es/motor/',
 'https://www.abc.es/sociedad',
 'https://www.abc.es/salud',
 'https://www.abc.es/natural/',
 'https://www.abc.es/ciencia/',
 'https://www.abc.es/tecnologia',
 'https://www.abc.es/cultura',
 'https://www.abc.es/historia',
 'https://www.abc.es/deportes',
 'https://www.abc.es/estilo',
 'https://www.abc.es/viajar',
 'https://www.abc.es/gastronomia',
 'https://www.abc.es/familia',
 'https://www.abc.es/bienestar/',
 'https://www.abc.es/summum',
 'https://www.abc.es/gente',
 'https://www.abc.es/xlsemanal/',
 'https://www.abc.es/antropia/',
 'https://www.abc.es/voz/',
 'https://www.abc.es/ultimas-noticias/',
 'https://www.abc.es/traductor/',
 'https://www.abc.es/eltiempo/',
 'https://www.abc.es/concurso/',
 'https://www.lavanguardia.com/alminuto',
 'https://www.lavanguardia.com/internaci

In [493]:
submedia

'www.elpais.com'

In [494]:
sections

['https://www.elpais.com/internacional/',
 'https://www.elpais.com/opinion/',
 'https://www.elpais.com/espana/',
 'https://www.elpais.com/economia/',
 'https://www.elpais.com/sociedad/',
 'https://www.elpais.com/tecnologia/',
 'https://www.elpais.com/cultura/',
 'https://www.elpais.com/deportes/',
 'https://www.elpais.com/television/',
 'https://www.elpais.com/gente/',
 'https://www.elpais.com/gastronomia/',
 'https://www.elpais.com/ultimas-noticias/',
 'https://www.abc.es/opinion',
 'https://www.abc.es/espana',
 'https://www.abc.es/internacional',
 'https://www.abc.es/economia',
 'https://www.abc.es/motor/',
 'https://www.abc.es/sociedad',
 'https://www.abc.es/salud',
 'https://www.abc.es/ciencia/',
 'https://www.abc.es/tecnologia',
 'https://www.abc.es/cultura',
 'https://www.abc.es/deportes',
 'https://www.abc.es/gastronomia',
 'https://www.abc.es/gente',
 'https://www.abc.es/antropia/',
 'https://www.abc.es/ultimas-noticias/',
 'https://www.lavanguardia.com/internacional',
 'https:

In [532]:
len(unique_medias) // num_splits

8

In [535]:
sections_chunk

['https://www.deia.eus/economia/',
 'https://www.deia.eus/motogp/',
 'https://www.deia.eus/deportes/',
 'https://www.deia.eus/opinion/',
 'https://www.deia.eus/lo-ultimo/',
 'https://www.deia.eus/promociones/',
 'https://www.deia.eus/vivir/',
 'https://www.deia.eus/mapaweb/']

In [639]:
end

32

In [665]:
version_n = max(int(x.split("_")[-1][1:-4]) for x in glob.glob("./data/final_url_sections_v*.csv"))
sections_file_path = f"./data/final_url_sections_v{version_n}.csv"
num_splits = 2

sections = pd.read_csv(sections_file_path, 
                       sep=";")
medias = sections["media"].tolist()
unique_medias =  list(set(medias))
chunk_size = len(unique_medias) // num_splits
end = len(unique_medias)
remainder = len(unique_medias) % num_splits
if remainder > 0:
    end -= remainder
chunks = []
for i in range(0, end, chunk_size):
    if (i + chunk_size) < end:
        print("a", i, i + chunk_size, len(unique_medias))
        submedias = unique_medias[i:i + chunk_size]
    else:
        print("b", i, i + chunk_size + remainder, len(unique_medias))
        submedias = unique_medias[i:i + chunk_size + remainder]
    subchanks = []
    for submedia in submedias:
        sections_chunk = sections.loc[sections["media"] == submedia, "section"].tolist()
        subchanks.extend(sections_chunk)
    chunks.append(subchanks)

a 0 16 33
b 16 33 33


In [ ]:
sections = pd.read_csv(sections_file_path, 
                       sep=";")
medias = sections["media"].tolist()
unique_medias =  list(set(medias))
chunk_size = len(unique_medias) // num_splits
end = len(unique_medias)
remainder = len(unique_medias) % num_splits
if remainder > 0:
    end -= remainder
chunks = []
for i in range(0, end, chunk_size):
    if (i + chunk_size) < end:
        submedias = unique_medias[i:i + chunk_size]
    else:
        submedias = unique_medias[i:i + chunk_size + remainder]
    subchanks = []
    for submedia in submedias:
        sections_chunk = sections.loc[sections["media"] == submedia, "section"].tolist()
        subchanks.extend(sections_chunk)
    chunks.append(subchanks)

In [666]:
len(unique_medias)

33

In [667]:
chunk_size

16

In [668]:
len(sections)

376

In [669]:
len(unique_medias) // num_splits, len(unique_medias) % num_splits

(16, 1)

In [670]:
list(range(0, len(unique_medias), chunk_size))

[0, 16, 32]

In [672]:
len(chunks), len(chunks[0]) + len(chunks[1])

(2, 376)

In [576]:
unique_medias[i:i + chunk_size]

['www.diariovasco.com', 'www.cronicaglobal.elespanol.com', 'www.deia.eus']

In [572]:
remainder

3

In [569]:
unique_medias[i:i + chunk_size]

['www.sevilla.abc.es',
 'www.eldiariomontanes.es',
 'www.diariodecadiz.es',
 'www.diariodesevilla.es',
 'www.lavozdegalicia.es']

In [568]:
len(chunks)

6

In [566]:
list(range(0, len(unique_medias), chunk_size))

[0, 5, 10, 15, 20, 25, 30]

In [561]:
len(chunks)

7

In [ ]:
# Create a process for each chunk and run in parallel
processes = []
for i, chunk in enumerate(chunks):
    split_file_path = os.path.join("data", f"sections_split_{i}.txt")  
    with open(split_file_path, 'w') as split_file:
        split_file.write("\n".join(chunk))

In [537]:
sections_chunk

['https://www.deia.eus/economia/',
 'https://www.deia.eus/motogp/',
 'https://www.deia.eus/deportes/',
 'https://www.deia.eus/opinion/',
 'https://www.deia.eus/lo-ultimo/',
 'https://www.deia.eus/promociones/',
 'https://www.deia.eus/vivir/',
 'https://www.deia.eus/mapaweb/']

In [538]:
chunk_size

8

In [539]:
len(chunks)

5

#### Analysis of medias nodes

In [699]:
urls_serie = pd.Series(valid_links_complete_unique)
media_count = urls_serie.str.extract("[//]{2}(.*)[/]").value_counts()
media_count

www.sevilla.abc.es                     19
www.abc.es                             16
www.elplural.com                       13
www.lavanguardia.com                   10
www.huffingtonpost.es                   9
                                       ..
www.diariodesevilla.es/motociclismo     1
www.diariodesevilla.es/motor            1
www.diariodesevilla.es/mundo            1
www.diariodesevilla.es/ocio             1
www.vilaweb.cat/comunitat               1
Name: count, Length: 628, dtype: int64

In [676]:
media_nodes_count

opinion                        32
economia                       23
deportes                       23
sociedad                       22
motor                          18
                               ..
juan_manuel_marques_perales     1
invertia                        1
fernando_perez_avila            1
ana_s-_ameneiro                 1
cuadernos-del-sur               1
Name: count, Length: 375, dtype: int64

In [700]:
media_nodes = pd.Series(valid_links_complete_unique).str.replace("/{0,2}$", 
                                                                 "", 
                                                                 regex=True). \
                                                         str.extract(".*/{1}(.*)", 
                                                                     expand=False)
media_nodes_count = media_nodes.value_counts()
media_nodes_count_more_2 = media_nodes_count[media_nodes_count > 2]
#media_nodes_more_2_relfreq = (media_nodes_count_more_2 / media_nodes_count_more_2.sum())
#media_nodes_more_2_relfreq_smooth4 = media_nodes_more_2_relfreq.rolling(window=4, min_periods=1).mean()

common_sections = media_nodes_count_more_2.index.tolist()
filter_final_section_urls = links_serie.apply(lambda url: any(section in url for section in common_sections))
final_section_urls = links_serie[filter_final_section_urls]
final_section_urls = final_section_urls.str.extract("(?P<section>https?://(?P<media>[^/]+)/.*)")
final_section_urls

,section,media
0,https://www.diariocordoba.com/deportes/2023/08...,www.diariocordoba.com
1,https://www.diariocordoba.com/deportes/2023/08...,www.diariocordoba.com
3,https://www.diariocordoba.com/agricultura-medi...,www.diariocordoba.com
6,https://www.diariocordoba.com/cordoba-ciudad/2...,www.diariocordoba.com
7,https://www.diariocordoba.com/agricultura-medi...,www.diariocordoba.com
...,...,...
1242,NaN,NaN
1243,NaN,NaN
1247,NaN,NaN
1251,https://ocasion.neomotor.com/,ocasion.neomotor.com


In [704]:
(urls_serie == "").sum()

0

In [483]:
files_section = glob.glob("./data/final_url_sections_v*.csv")
if len(files_section) > 0:
    version_n = max(int(x.split("_")[-1][1:-4]) for x in files_section) + 1
else:
    version_n = 0
final_section_urls.to_csv(f"./data/final_url_sections_v{version_n}.csv", 
                          index=False, 
                          header=True,
                          sep=";"
                         )
glob.glob("./final_url_sections_v*.csv")

[]

In [45]:
from nltk.corpus import wordnet as wn
from nltk import download
from nltk.metrics.distance import edit_distance
#download('wordnet')

In [46]:
edit_distance()

<function nltk.metrics.distance.edit_distance(s1, s2, substitution_cost=1, transpositions=False)>

In [48]:
words = media_nodes.tolist()

In [54]:
processed = []
scores = {}
for w1 in tqdm.tqdm(words):
        processed.append(w1)
        for w2 in words:
            if w2 not in processed:
                scores[(w1, w2)] = edit_distance(w1, w2)

  8%|██████▎                                                                      | 198/2403 [08:40<1:36:34,  2.63s/it]


KeyboardInterrupt: 

In [69]:
scores

{('internacional', 'internacional'): 0,
 ('internacional', 'opinion'): 9,
 ('internacional', 'espana'): 9,
 ('internacional', 'economia'): 10,
 ('internacional', 'sociedad'): 10,
 ('internacional', 'educacion'): 8,
 ('internacional', 'tecnologia'): 9,
 ('internacional', 'cultura'): 11,
 ('internacional', 'deportes'): 12,
 ('internacional', 'television'): 9,
 ('internacional', 'defensor-a-del-lector'): 17,
 ('internacional', 'gente'): 11,
 ('internacional', 'podcasts'): 12,
 ('internacional', 'gastronomia'): 10,
 ('internacional', 'mamas-papas'): 12,
 ('internacional', 'america-futura'): 12,
 ('internacional', 'escaparate'): 11,
 ('internacional', 'ultimas-noticias'): 12,
 ('internacional', 'especiales'): 10,
 ('internacional', 'elecciones-generales-23j'): 20,
 ('internacional', 'motor'): 11,
 ('internacional', 'salud'): 12,
 ('internacional', 'natural'): 9,
 ('internacional', 'ciencia'): 8,
 ('internacional', 'historia'): 9,
 ('internacional', 'archivo'): 11,
 ('internacional', 'estilo

In [88]:
sorted(scores.items(), key=lambda k: (k[0][0], k[1]), reverse=False)

[(('abceduca', 'abceduca'), 0),
 (('abceduca', 'faroeduca'), 3),
 (('abceduca', 'cerca'), 4),
 (('abceduca', 'coruna'), 5),
 (('abceduca', 'arousa'), 5),
 (('abceduca', 'almeria'), 5),
 (('abceduca', 'antequera'), 5),
 (('abceduca', 'agenda'), 5),
 (('abceduca', 'blanca'), 5),
 (('abceduca', 'abanilla'), 5),
 (('abceduca', 'aledo'), 5),
 (('abceduca', 'archena'), 5),
 (('abceduca', 'ceuti'), 5),
 (('abceduca', 'huesca'), 5),
 (('abceduca', 'chance'), 6),
 (('abceduca', 'valencia'), 6),
 (('abceduca', 'amarina'), 6),
 (('abceduca', 'deza'), 6),
 (('abceduca', 'cdlugo'), 6),
 (('abceduca', 'bizkaia'), 6),
 (('abceduca', 'alava'), 6),
 (('abceduca', 'comics'), 6),
 (('abceduca', 'agencias'), 6),
 (('abceduca', 'ribera'), 6),
 (('abceduca', 'athletic'), 6),
 (('abceduca', 'mallorca'), 6),
 (('abceduca', 'caceres'), 6),
 (('abceduca', 'merida'), 6),
 (('abceduca', 'elda'), 6),
 (('abceduca', 'alcoy'), 6),
 (('abceduca', 'elche-cf'), 6),
 (('abceduca', 'cadiz'), 6),
 (('abceduca', 'sierra'),

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\marsu\AppData\Roaming\nltk_data...


True

In [25]:
file_name = "spain_media name_to_url.json"
file_path = os.path.join(".", "data", file_name)
if os.path.exists(file_path):
    print("Reading file of urls of regions...")
    name_to_media_urls = read_media_urls_file(file_path)
for (_, media_url) in name_to_media_urls.items():
    try:
        response = requests.get(media_url, headers=HEADERS, timeout=10)  # Set an appropriate timeout value (in seconds)
    except requests.exceptions.Timeout:
        # Handle the timeout exception
        print("The request timed out.")
        ErrorReporter(f"Get request timeout exception at {media_url}")
        return []
    except requests.exceptions.RequestException as e:
        # Handle other request exceptions
        print(f"An error occurred: {str(e)}")
        ErrorReporter(f"Get request general exception, {e}, at {media_url}")
        return []

    parsed_hmtl = BeautifulSoup(response.content, "html.parser")
    #for x in parsed_hmtl.body.find_all("a"):
    #    x.attrs.get("href", None)
    try:
        links = [x.attrs.get("href", None) for x in parsed_hmtl.body.find_all("a")]
    except Exception as e:
        print(e)
        return None

    links_serie = pd.Series(links).dropna()
    nodes = links_serie.str.replace(media_url, "", regex=True)

    nodes_split = nodes.str.split("/")
    nodes_split_clean = nodes_split.apply(lambda x: [elem for elem in x if elem])

    nodes_split_clean_filter = nodes_split_clean.str.len().eq(1)

    valid_links = links_serie[nodes_split_clean_filter]
    valid_links_complete = [media_url + x if not x.startswith(media_url) else x for x in valid_links]
    valid_links_complete = []
    with open("Skipped urls.txt", "w") as file_skipped:
        for link_node in valid_links:
            link = link_node
            if not link.startswith(media_url):
                if link.startswith("https://www."):
                    print(f"{link} skipped. find_media_section(). continue_1 ", file=file_skipped)
                    continue
                else:
                    link_parts = link.split("/")
                    link_parts = [x for x in link_parts if x]
                    if len(link_parts) == 1:
                        if link.startswith("/"):
                            link = media_url + link
                        else:
                            link = media_url + "/" + link
                    else:
                        continue
            else:
                link_parts = link.split("/")
                link_parts = [x for x in link_parts if x][2:]
                if len(link_parts) == 1:
                    if link.startswith("/"):
                        link = media_url + link
                    else:
                        link = media_url + "/" + link
                else:
                    continue

            """link = link_node
            # base url is not in link node
            if not link_node.startswith(media_url):
                # www is in link node (not valid main url)
                if link_node.startswith("https://www."):
                    print(f"{link_node} skipped. find_media_section(). continue_1 ", file=file_skipped)
                    continue
                # Valid url node
                elif not link_node.startswith("/"):
                    link = media_url + "/" + link
                else:
                    link = media_url + link"""
                
            valid_links_complete.append(link)
    valid_links_complete_unique = pd.Series(valid_links_complete).unique().tolist()

    for media_section_url in valid_links_complete_unique:
        try:
            response = requests.get(media_url, headers=HEADERS, timeout=10)  # Set an appropriate timeout value (in seconds)
        except requests.exceptions.Timeout:
            # Handle the timeout exception
            print("The request timed out.")
            ErrorReporter(f"Get request timeout exception at {media_url}")
            return [], on_start_checkpoint
        except requests.exceptions.RequestException as e:
            # Handle other request exceptions
            print(f"An error occurred: {str(e)}")
            ErrorReporter(f"Get request general exception, {e}, at {media_url}")
            return [], on_start_checkpoint

        parsed_hmtl = BeautifulSoup(response.content, "html.parser")

        tags_with_url = parsed_hmtl.find_all("a", href=re.compile("https?:.*"))

        tags_with_url = pd.Series(tags_with_url).unique().tolist()

        valid_news_urls = []
        with open("Skipped urls.txt", "w") as file_skipped:
            for i, tag_with_url in enumerate(tags_with_url):
                news_url = tag_with_url.attrs["href"]
                # Filter out urls with query symbols
                if re.search("[?=&+%@#]{1}", news_url):
                    search_spec_char = re.search("[?=&+%@#]{1}", news_url)
                    query_start = search_spec_char.span()[0]
                    news_url = news_url[:query_start]
                #if not re.search("/(\D+-+\D+-?)+/?", news_url) and "html" not in news_url:
                #    print(f"{news_url} skipped. find_news_urls(). continue_1", file="Skipped urls.txt")
                #    continue
                if news_url.endswith("/"):
                    print(f"{news_url} skipped. find_news_urls(). continue_2", file=file_skipped)
                    continue
                if re.search("https?:[\/]{2}", news_url):
                    if media_url not in news_url:
                        print(f"{news_url} skipped. find_news_urls(). continue_3", file=file_skipped)
                        continue
                if media_url in news_url or media_url.replace("https://www.", "") in news_url or media_url.replace("http://www.", "") in news_url:
                    if on_start_checkpoint:
                        if news_url == last_news_url or not last_news_url:
                            on_start_checkpoint = False
                        else:
                            print(f"{news_url} skipped skip. find_news_urls(). continue_4 ", file=file_skipped)
                            continue
                    valid_news_urls.append(news_url)
                        
        urls = list(set(valid_news_urls))

Reading file of urls of regions...


SyntaxError: 'return' outside function (85991022.py, line 13)

In [26]:
    news_media_data = []
    today_iso = datetime.today().isoformat(sep=" ")
    for news_url in urls:
        print("Finding data for url:", news_url)
        resp_url_news = requests.get(news_url, 
                                     headers=HEADERS)
        parsed_news_hmtl = BeautifulSoup(resp_url_news.content, 
                                         "html.parser")
        # Accept or reject url if news date is more than two days older
        try:
            time_tag = parsed_news_hmtl.select("time")[0]
            timestamp = time_tag.attrs["datetime"]
            timestamp_minus_2days = datetime.fromisoformat(timestamp) - timedelta(days=N_MAX_DAYS_OLD)
            if timestamp_minus_2days < today_iso:
                print(f"Old {news_url}")
                continue
        except:
            pass

NameError: name 'urls' is not defined

In [9]:
url = "https://www.eldiariomontanes.es/economia"
[x for x in url.split("/") if x][2:]

['economia']

In [14]:
url1 = "https://www.france24.com/en/live-news/20230818-drug-war-bloodbath-shakes-marseille-as-rival-gangs-shoot-it-out-1"
url2 = "https://www.huffingtonpost.es/politica/campana-feijoo-pese-ganar-14700-euros-mes-politica-atesorar-patrimonio.html"
url3 = "https://www.lavanguardia.com/economia/20230819/9174627/alimentos-ponen-tension-objetivos-inflacion-zona-euro.html"
url4 = "https://www.xataka.com/basics/como-declarar-a-hacienda-las-inversiones-en-bitcoin-y-criptomonedas-con-beneficios"
url5 = "https://www.elconfidencial.com/"
url6 = "https://www.cronicaglobal.elespanol.com/".replace("www.", "")
url7 = "https://www.elespanol.com/"
url8 = "https://www.elespanol.com/opinion/editoriales/20230822/machismo-rubiales-convierte-posicion-insostenible/788561138_14.html"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36"
}
response = requests.get(url7, 
                        headers=HEADERS)
parsed_html = BeautifulSoup(response.content, 
                            "html.parser")
hrefs = parsed_html.html.body.find_all("a", href=re.compile(".*"))
hrefs

[<a href="https://www.elespanol.com/opinion/editoriales/20230822/machismo-rubiales-convierte-posicion-insostenible/788561138_14.html">El Rugido</a>,
 <a href="https://www.elespanol.com/ciencia/salud/20230822/suben-contagios-vuelven-mascarillas-hospitales-pasando-covid/788421503_0.html">Qué pasa con la COVID</a>,
 <a href="https://www.elespanol.com/reportajes/20230822/eloy-asesino-exmilitar-first-dates-mata-punaladas-hombre-bar-cordoba/788421541_0.html">Eloy, el asesino exmilitar de First Dates</a>,
 <a href="https://www.elespanol.com/deportes/futbol/20230821/apoteosis-futbol-femenino-mundial-marca-hito-sociedad-espanola/788171293_0.html">Apoteosis del fútbol femenino</a>,
 <a href="https://www.elespanol.com/deportes/futbol/20230820/madre-oculto-olga-carmona-muerte-padre-entero-volaba-sidney/788171310_0.html">La madre ocultó a Olga Carmona la muerte de su padre</a>,
 <a href="https://www.elespanol.com/deportes/futbol/20230821/reacciones-beso-rubiales-jenni-hermoso-en-directo-muere-padre

In [31]:
url = "https://www.canarias7.es/cultura/musica-d/"
#url = "https://www.diariodemallorca.es/ideas/el-futbol/somos-mas.html"
#url = "https://www.canarias7.es/temas/generales/fiestas-dsd-populares.html"
#url = "https://www.abc.es/motor/motos/"
#url = "https://www.abc.es/motor/novedades/"
#url = "https://www.huffingtonpost.es/static/quienes.html"
url = "https://www.europapress.es/galicia/agro-00246/noticia-xunta-otorgara-ayudas-recuperacion-castanos-afectados-incendios-forestales-galicia-20230821130636.html"
url = "https://www.huffingtonpost.es/static/politica-de-cookies.html"
url = "https://www.diariosur.es/temas/generales/sequia.html"
url = "https://www.elespanol.com/mundo/792/"
url = "https://www.elcorreo.com/temas/generales/antropia-igualdad-de-genero.html"
#url = "https://www.elcorreo.com/especial/obituario-bizkaia-2023/"
#url = "https://www.diariovasco.com/mi-seccion/"
#url = "https://www.lavanguardia.com/deportes/resultados/ligue-1"
#url = "https://www.levante-emv.com/tour-francia/dsdsd-dsddddsddsd"
#url = "https://www.diariodemallorca.es/salud/cuidamos-tu-salud/mujer-y-embarazo/"
url = "https://www.levante-emv.com/ocio/agenda/eventos/exposiciones_c/valencia_pr/"
url = "https://www.larioja.com/mi-seccion/"
url = "https://www.diariovasco.com/mi-seccion/"
url = "https://www.elespanol.com/espana/politica/20230821/vox-no-garantiza-apoyo-feijoo-cordon-sanitario-mesa-exige-explicaciones/788421341_0.html"
url = "https://www.diariovasco.com/pantallas/series/series-verano-refrescantes-20230820004311-ntrc.html"
url = "https://www.deia.eus/aste-nagusia/2023/08/21/tres-detenidos-tocamientos-segunda-jornada-7168039.html"
url = "https://www.deia.eus/aste-nagusia/2023/08/21/tres-detenidos-tocamientos-segunda-jornada-7168039.html"
url = "https://www.elespanol.com/opinion/editoriales/20230822/machismo-rubiales-convierte-posicion-insostenible/788561138_14.html"
url = "https://www.noticiasdenavarra.com/politica/2023/08/22/sanchez-ofrece-rey-investidura-ve-7171824.html"
news_url_splits =  [x for x in url.split("/") if x][2:]
news_url_splits, len(news_url_splits), news_url_splits[-1]

(['politica',
  '2023',
  '08',
  '22',
  'sanchez-ofrece-rey-investidura-ve-7171824.html'],
 5,
 'sanchez-ofrece-rey-investidura-ve-7171824.html')

In [33]:
[x for x in url8.split("/") if x][2:][-1]

'788561138_14.html'

In [34]:
re.search(r"^[.a-zA-Z0-9]+(-[.a-zA-Z0-9]+){,2}$", news_url_splits[-1])

In [175]:
import time

In [22]:
s = time.time()
re.search(r"^[._a-zA-Z0-9]+(-[._a-zA-Z0-9]+){,2}$", news_url_splits[-1])
time.time() - s

NameError: name 'time' is not defined

In [26]:
json.loads(None, strict=False)

TypeError: the JSON object must be str, bytes or bytearray, not NoneType

### Read and review sections media get requests

In [233]:
os.listdir("../data")

['extraction checkpoint.json',
 'extraction checkpoint_0.json',
 'extraction checkpoint_1.json',
 'extraction checkpoint_10.json',
 'extraction checkpoint_11.json',
 'extraction checkpoint_2.json',
 'extraction checkpoint_3.json',
 'extraction checkpoint_4.json',
 'extraction checkpoint_5.json',
 'extraction checkpoint_6.json',
 'extraction checkpoint_7.json',
 'extraction checkpoint_8.json',
 'extraction checkpoint_9.json',
 'extraction-errors.txt',
 'final_url_sections_v3.csv',
 'final_url_sections_v4.csv',
 'final_url_sections_v5.csv',
 'final_url_sections_v6.csv',
 'full_news_table_02 09 2023.csv',
 'news_db.sqlite3',
 'plain_spain_media name_to_url.txt',
 'region_to_media_urls.json',
 'role-prompt-body-extraction.txt',
 'role-prompt-body-extraction_paragraphs.txt',
 'role-prompt-keys-extraction.txt',
 'SCImago Media Ranking - Spain - Spanish_02092023.xlsx',
 'SCImago Media Ranking - Spain - Spanish_20082023.xlsx',
 'scrapping-statistics',
 'spain_audience_total_traffic_2023-07-09 

In [272]:
sections = pd.read_csv("../data/final_url_sections_v6.csv", 
                       header=0, 
                       sep=";").sort_values("section")
sections

,section,media
344,https://cronicaglobal.elespanol.com/politica/,cronicaglobal.elespanol.com/
209,https://www.abc.es/antropia/,www.abc.es/
341,https://www.abc.es/ciencia/,www.abc.es/
142,https://www.abc.es/cultura/,www.abc.es/
131,https://www.abc.es/deportes/,www.abc.es/
...,...,...
62,https://www.sevilla.abc.es/salud/,www.sevilla.abc.es/
70,https://www.sevilla.abc.es/sociedad/,www.sevilla.abc.es/
262,https://www.sevilla.abc.es/tecnologia/,www.sevilla.abc.es/
19,https://www.sevilla.abc.es/ultimas-noticias/,www.sevilla.abc.es/


In [275]:
differences = {}
for (idx, values) in sections.iterrows():
    section = values["section"]
    try:
        response = requests.get(values["section"], 
                                headers=HEADERS,
                                timeout=3.0
                               )
    except:
        print("error:", section)
    if section != response.url:
        print("\t", values["section"], " - ", response.url, " - ", values["media"], end="\n\n")
        differences[section] = response.url
differences

error: https://www.eldia.es/ocio/
	 https://www.eldia.es/ocio/  -  https://www.eldia.es/nacional/  -  www.eldia.es/

	 https://www.elpais.com/cultura/  -  https://elpais.com/cultura/  -  www.elpais.com/

	 https://www.elpais.com/deportes/  -  https://elpais.com/deportes/  -  www.elpais.com/

	 https://www.elpais.com/economia/  -  https://elpais.com/economia/  -  www.elpais.com/

	 https://www.elpais.com/espana/  -  https://elpais.com/espana/  -  www.elpais.com/

	 https://www.elpais.com/gastronomia/  -  https://elpais.com/gastronomia/  -  www.elpais.com/

	 https://www.elpais.com/gente/  -  https://elpais.com/gente/  -  www.elpais.com/

	 https://www.elpais.com/internacional/  -  https://elpais.com/internacional/  -  www.elpais.com/

	 https://www.elpais.com/sociedad/  -  https://elpais.com/sociedad/  -  www.elpais.com/

	 https://www.elpais.com/tecnologia/  -  https://elpais.com/tecnologia/  -  www.elpais.com/

	 https://www.elpais.com/television/  -  https://elpais.com/television/  -

{'https://www.eldia.es/ocio/': 'https://www.eldia.es/nacional/',
 'https://www.elpais.com/cultura/': 'https://elpais.com/cultura/',
 'https://www.elpais.com/deportes/': 'https://elpais.com/deportes/',
 'https://www.elpais.com/economia/': 'https://elpais.com/economia/',
 'https://www.elpais.com/espana/': 'https://elpais.com/espana/',
 'https://www.elpais.com/gastronomia/': 'https://elpais.com/gastronomia/',
 'https://www.elpais.com/gente/': 'https://elpais.com/gente/',
 'https://www.elpais.com/internacional/': 'https://elpais.com/internacional/',
 'https://www.elpais.com/sociedad/': 'https://elpais.com/sociedad/',
 'https://www.elpais.com/tecnologia/': 'https://elpais.com/tecnologia/',
 'https://www.elpais.com/television/': 'https://elpais.com/television/',
 'https://www.elpais.com/ultimas-noticias/': 'https://elpais.com/ultimas-noticias/',
 'https://www.elplural.com/economia/': 'https://www.elplural.com/economia',
 'https://www.elplural.com/politica/': 'https://www.elplural.com/politic

In [ ]:
response.cookies.